# 🐄 Sistema de Estimación de Peso Bovino - Setup ML

> **BLOQUE 0**: Información del proyecto (markdown - solo lectura)

**Proyecto**: Hacienda Gamelera - Bruno Brito Macedo  
**Responsable**: Persona 2 - Setup Infraestructura ML  
**Objetivo**: Preparar datasets y pipeline para entrenamiento de 7 modelos (razas tropicales priorizadas)  
**Duración**: 5-6 días  

---

## 📑 Índice de Bloques (Referencia Rápida)

| Bloque | Nombre | Descripción | Requisitos |
|--------|--------|-------------|------------|
| **0** | Información | Markdown introductorio | Ninguno |
| **1** | Clonar Repositorio | Monta Drive y clona desde GitHub (persistente) | Ninguno (requiere internet) |
| **2** | Verificar Dependencias | Verifica versiones base de TensorFlow y NumPy | Ninguno |
| **3** | Instalar Dependencias Críticas | TensorFlow 2.19.0, NumPy 2.x, MLflow, DVC | Ninguno |
| **4** | Instalar Complementos | Albumentations, OpenCV, herramientas ML | Ninguno |
| **5** | Configuración Proyecto | Crea estructura de carpetas y variables globales | Bloque 1 |
| **6** | Descargar Imágenes Propias | Scraping de imágenes para dataset personalizado | Bloque 5 |
| **7** | Descargar CID Dataset | Descarga CID Dataset desde S3 (complementario) | Bloque 5 |
| **8** | Preparar Dataset Combinado | Combina CID + Nuestras imágenes (Estrategia B) | Bloques 6 + 7 |
| **9** | Resumen Datasets | Muestra resumen de datasets disponibles (CID + propias) | Bloques 6-8 |
| **10** | Verificación Rápida | Verificación mínima de columnas requeridas (OPCIONAL) | Bloque 9 |
| **11** | Pipeline de Datos | Pipeline con augmentation | Bloque 8 |
| **12** | Arquitectura Modelo | Crea modelo EfficientNetB0 | Bloque 11 |
| **13** | Configurar Entrenamiento | Callbacks y MLflow | Bloque 12 |
| **14** | Entrenamiento | Entrena modelo (2-4h) | Bloque 13 + GPU |
| **15** | Evaluación | Evalúa modelo | Bloque 14 |
| **16** | Exportar TFLite | Exporta modelo a TFLite | Bloque 15 |

---

## 📋 Flujo de Trabajo

### Día 1: Setup (Bloques 1-5)
- **BLOQUE 1**: Clonar repositorio en Drive
- **BLOQUE 2**: Verificar dependencias base
- **BLOQUE 3**: Instalar dependencias críticas (TensorFlow, MLflow)
- **BLOQUE 4**: Instalar complementos (Albumentations, OpenCV)
- **BLOQUE 5**: Configurar proyecto y carpetas

### Día 2-3: Datasets (Bloques 6-9)
- **BLOQUE 6**: Descargar nuestras imágenes (razas bolivianas, etapas de crianza)
- **BLOQUE 7**: Descargar CID Dataset (complementario - 17,899+ imágenes)
- **BLOQUE 8**: Preparar dataset combinado (Estrategia B: combina CID + nuestras imágenes)
- **BLOQUE 9**: Resumen de datasets disponibles (verifica combinación)

### Día 4: Verificación (Bloque 10) - OPCIONAL
- **BLOQUE 10**: Verificación rápida de datos (solo comprueba columnas necesarias, sin gráficos)
- 💡 **NOTA**: Este bloque es OPCIONAL. Puedes saltarlo para entrenar más rápido.

### Día 5-6: Pipeline y Modelo (Bloques 11-16)
- **BLOQUE 11**: Pipeline de datos con augmentation (usa dataset combinado)
- **BLOQUE 12**: Arquitectura del modelo
- **BLOQUE 13**: Configurar entrenamiento
- **BLOQUE 14**: Entrenar modelo
- **BLOQUE 15**: Evaluación del modelo
- **BLOQUE 16**: Exportar a TFLite

---

## 🎯 Razas Objetivo (7 razas)
1. **Nelore** – Carne tropical dominante en Santa Cruz (≈42% del hato)
2. **Brahman** – Cebuino versátil para cruzamientos y climas extremos
3. **Guzerat** – Doble propósito (carne/leche) con gran rusticidad materna
4. **Senepol** – Carne premium adaptada al calor, ideal para “steer” de alta calidad
5. **Girolando** – Lechera tropical (Holstein × Gyr) muy difundida en sistemas semi-intensivos
6. **Gyr lechero** – Lechera pura clave para genética tropical y sólidos altos
7. **Sindi** – Lechera tropical compacta, de alta fertilidad y leche rica en sólidos

> Estas razas cubren el portafolio real de Santa Cruz (carne tropical + lecheras adaptadas). Más adelante podemos sumar Holstein, Pardo Suizo o Jersey si obtenemos datos suficientes.


In [2]:
# ============================================================
# BLOQUE 1: CONFIGURAR RUTA DEL PROYECTO Y CLONAR REPOSITORIO EN DRIVE
# ============================================================
# 📁 Clona el repositorio desde GitHub a Google Drive (persistente entre sesiones)
# 🔗 Repositorio: https://github.com/Angello-27/bovine-weight-estimation.git
# 💾 Se clona en Drive para que persista entre desconexiones del runtime

import sys
import subprocess
from pathlib import Path

GITHUB_REPO_URL = 'https://github.com/Angello-27/bovine-weight-estimation.git'

# Montar Google Drive (solo si no está montado)
print("🔗 Verificando Google Drive...")
try:
    from google.colab import drive
    drive_path = Path('/content/drive')
    if not drive_path.exists() or not any(drive_path.iterdir()):
        drive.mount('/content/drive')
    BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')
except ImportError:
    BASE_DIR = Path('/content/bovine-weight-estimation')

print(f"📁 Directorio base: {BASE_DIR}")

# Clonar o sincronizar repositorio
if BASE_DIR.exists() and (BASE_DIR / '.git').exists():
    print("🔄 Sincronizando repositorio existente...")
    subprocess.run(['git', 'pull'], cwd=str(BASE_DIR), check=False)
else:
    print("📥 Clonando repositorio...")
    BASE_DIR.parent.mkdir(parents=True, exist_ok=True)
    result = subprocess.run(['git', 'clone', GITHUB_REPO_URL, str(BASE_DIR)], check=False)
    if result.returncode != 0:
        raise RuntimeError(f"Error al clonar repositorio. Verifica conexión a internet.")

# Configurar PYTHONPATH
ML_TRAINING_DIR = BASE_DIR / 'ml-training'
src_dir = ML_TRAINING_DIR / 'src'
if src_dir.exists():
    sys.path.insert(0, str(src_dir))
    print(f"✅ PYTHONPATH configurado: {src_dir}")
else:
    print(f"⚠️ Directorio src no encontrado: {src_dir}")

print(f"✅ Configuración completada")
print(f"📁 ML Training: {ML_TRAINING_DIR}")


## 🚀 Día 1: Setup Google Colab Pro + Dependencias

In [ ]:
# ============================================================
# BLOQUE 2: VERIFICACIÓN DE DEPENDENCIAS BASE
# ============================================================
# 🔍 Verifica versiones base de TensorFlow y NumPy
# 💡 Solo verifica - no desinstala (pip maneja versiones automáticamente)
# ⚠️ Si hay conflictos, ejecuta limpieza manual o reinicia el runtime

import warnings
warnings.filterwarnings('ignore')

print("🔍 VERIFICANDO DEPENDENCIAS BASE...\n")

# Verificar versiones actuales
try:
    import tensorflow as tf
    import numpy as np
    print(f"📦 Versiones actuales:")
    print(f"   - TensorFlow: {tf.__version__}")
    print(f"   - NumPy: {np.__version__}")
    
    # Verificar compatibilidad básica
    tf_ok = tf.__version__.startswith('2.')
    numpy_ok = np.__version__.startswith('2.') or np.__version__.startswith('1.')
    
    if tf_ok and numpy_ok:
        print(f"\n✅ Versiones compatibles detectadas")
        print(f"💡 Continúa con el BLOQUE 3 para instalar dependencias")
    else:
        print(f"\n⚠️ Versiones pueden tener conflictos")
        print(f"💡 Recomendación: Reinicia el runtime o ejecuta limpieza manual")
        
except ImportError as e:
    print(f"⚠️ Error importando dependencias: {e}")
    print(f"💡 Esto es normal en un runtime nuevo - continúa con el BLOQUE 3")

print(f"\n💡 NOTA: El BLOQUE 3 instalará las versiones correctas automáticamente")
print(f"💡 No es necesario desinstalar/reinstalar manualmente si las versiones están bien definidas")


In [ ]:
# ============================================================
# BLOQUE 3: INSTALACIÓN DE DEPENDENCIAS CRÍTICAS
# ============================================================
# 🔧 Instala dependencias críticas con versiones exactas
# ✅ pip maneja automáticamente las actualizaciones

import warnings
import subprocess
warnings.filterwarnings('ignore')

print("📦 INSTALANDO DEPENDENCIAS CRÍTICAS...\n")

# Instalar dependencias en orden
dependencies = [
    ("tf-keras", "tf-keras>=2.19.0"),
    ("packaging", "packaging<25"),
    ("wrapt", "wrapt<2.0.0,>=1.10.10"),
    ("requests", "requests==2.32.4"),
    ("jedi", "jedi>=0.16"),
    ("MLflow", "mlflow==2.16.2"),
    ("DVC", "dvc[gs,s3]==3.51.1"),
    ("scikit-learn", "scikit-learn>=1.6")
]

for name, package in dependencies:
    result = subprocess.run(
        ['pip', 'install', '-q', '--no-cache-dir', package],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print(f"✅ {name} instalado")
    else:
        print(f"⚠️ Error instalando {name}")

# Configurar mixed precision
try:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print("✅ Mixed precision (FP16) activado\n")
except Exception as e:
    print(f"⚠️ Mixed precision no disponible: {str(e)[:50]}\n")

# Verificar instalaciones
import tensorflow as tf
import numpy as np

print("=" * 60)
print("✅ DEPENDENCIAS CRÍTICAS INSTALADAS")
print("=" * 60)

versions = {
    'TensorFlow': tf.__version__,
    'NumPy': np.__version__
}

packages_to_check = {
    'MLflow': ('mlflow', '__version__'),
    'Scikit-learn': ('sklearn', '__version__'),
    'Protobuf': ('google.protobuf', '__version__'),
    'ml_dtypes': ('ml_dtypes', '__version__'),
    'tf-keras': ('tensorflow.keras', '__version__')
}

for name, version in versions.items():
    print(f"   - {name}: {version}")

for name, (module, attr) in packages_to_check.items():
    try:
        mod = __import__(module, fromlist=[attr])
        version = getattr(mod, attr) if hasattr(mod, attr) else getattr(mod, '__version__', 'OK')
        print(f"   - {name}: {version} ✅")
    except Exception:
        print(f"   - {name}: No disponible ⚠️")

# Verificación de compatibilidad
tf_ok = tf.__version__.startswith('2.19')
numpy_ok = np.__version__.startswith('2.0') or np.__version__.startswith('2.1')

print(f"\n🔍 Compatibilidad: TF 2.19.x {'✅' if tf_ok else '⚠️'}, NumPy 2.x {'✅' if numpy_ok else '⚠️'}")

if tf_ok and numpy_ok:
    print("\n✅ Instalación completada exitosamente")
else:
    print("\n⚠️ Verifica versiones instaladas")


In [ ]:
# ============================================================
# BLOQUE 4: INSTALACIÓN DE COMPLEMENTOS
# ============================================================
# 🔧 Instala complementos: Albumentations, OpenCV, herramientas ML

import warnings
import subprocess
warnings.filterwarnings('ignore')

print("📦 INSTALANDO COMPLEMENTOS...\n")

# Instalar complementos
complements = [
    ("albumentations>=2.0.8", "Albumentations"),
    ("gdown", "gdown"),
    ("plotly", "Plotly"),
    ("seaborn", "Seaborn"),
    ("pillow>=11.0.0", "Pillow")
]

for package, name in complements:
    result = subprocess.run(
        ['pip', 'install', '-q', '--no-cache-dir', package],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print(f"✅ {name} instalado")
    else:
        print(f"⚠️ Error instalando {name}")

# Verificar/instalar OpenCV
try:
    import cv2
    print(f"✅ OpenCV {cv2.__version__} disponible")
except ImportError:
    result = subprocess.run(
        ['pip', 'install', '-q', '--no-cache-dir', 'opencv-python-headless'],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        import cv2
        print(f"✅ OpenCV {cv2.__version__} instalado")
    else:
        print(f"⚠️ Error instalando OpenCV")

# Verificar e importar
import numpy as np
import tensorflow as tf

try:
    import cv2
    import albumentations as A
    import sklearn
    print(f"\n✅ Complementos verificados: OpenCV {cv2.__version__}, Albumentations {A.__version__}")
except Exception as e:
    print(f"\n⚠️ Algunos complementos no disponibles: {str(e)[:50]}")

# Configurar GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"\n✅ GPU configurada: {len(gpus)} dispositivo(s)")
        for i, gpu in enumerate(gpus):
            print(f"   - GPU {i}: {gpu.name}")
    except RuntimeError as e:
        print(f"\n⚠️ Error configurando GPU: {str(e)[:50]}")
else:
    print(f"\n⚠️ GPU no detectada - Activa en: Entorno > Cambiar tipo > GPU")

print("\n✅ Instalación de complementos completada")


In [ ]:
# ============================================================
# BLOQUE 5: CONFIGURACIÓN DEL PROYECTO Y ESTRUCTURA DE CARPETAS
# ============================================================
# ⚙️ Crea estructura de carpetas y configura variables globales

from pathlib import Path

# Verificar y montar Google Drive si es necesario
print("🔗 Verificando Google Drive...")
try:
    from google.colab import drive
    drive_path = Path('/content/drive')
    if not drive_path.exists() or not any(drive_path.iterdir()):
        print("📥 Montando Google Drive...")
        drive.mount('/content/drive')
        print("✅ Google Drive montado")
    else:
        print("✅ Google Drive ya está montado")
except ImportError:
    print("⚠️ No se detectó Google Colab - continuando sin Drive")
except Exception as e:
    print(f"⚠️ Error al montar Drive: {e}")
    print("💡 Asegúrate de autorizar el acceso a Google Drive")

# Verificar proyecto
BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')
if not BASE_DIR.exists():
    raise RuntimeError(f"Proyecto no encontrado en {BASE_DIR}. Ejecuta el bloque anterior primero.")

print(f"✅ Proyecto: {BASE_DIR}")

# Crear estructura de carpetas
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
AUGMENTED_DIR = DATA_DIR / 'augmented'
MODELS_DIR = BASE_DIR / 'models'
MLRUNS_DIR = BASE_DIR / 'mlruns'

for dir_path in [DATA_DIR, RAW_DIR, PROCESSED_DIR, AUGMENTED_DIR, MODELS_DIR, MLRUNS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Configurar MLflow
try:
    import mlflow
    mlflow.set_tracking_uri(f"file://{MLRUNS_DIR}")
    mlflow.set_experiment("bovine-weight-estimation")
    mlflow_available = True
except ImportError:
    print("⚠️ MLflow no disponible - ejecuta bloque de instalación")
    mlflow_available = False

# Configuración de entrenamiento
CONFIG = {
    'image_size': (224, 224),
    'batch_size': 32,
    'epochs': 100,
    'learning_rate': 0.001,
    'validation_split': 0.2,
    'test_split': 0.1,
    'early_stopping_patience': 10,
    'target_r2': 0.95,
    'max_mae': 5.0,
    'max_inference_time': 3.0
}

# Razas objetivo
BREEDS = [
    'brahman', 'nelore', 'angus', 'cebuinas',
    'criollo', 'pardo_suizo', 'guzerat', 'holstein'
]

print(f"✅ Configuración completada")
print(f"📁 Carpetas creadas: data/, models/, mlruns/")
print(f"🎯 Razas: {len(BREEDS)} razas")
if mlflow_available:
    print(f"📊 MLflow: {MLRUNS_DIR} ✅")


## 📥 Día 2-3: Descargar y Organizar Datasets Críticos


In [ ]:
# ============================================================
# BLOQUE 6: SCRAPING SIMPLE
# ============================================================
# 🎯 Descarga imágenes para todas las razas definidas
# 📋 Razas: Nelore, Brahman, Guzerat, Senepol, Girolando, Gyr lechero, Sindi
# 🧮 Genera metadata_estimada.csv (weight_in_kg, breed, life_stage) alineada con CID

import subprocess
import shutil
import random
import pandas as pd
from pathlib import Path

print("=" * 60)
print("📥 BLOQUE 6: DESCARGAR IMÁGENES")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

print(f"📁 RAW_DIR: {RAW_DIR}")

random.seed(42)
SCRAPED_DIR = RAW_DIR / 'scraped'
SCRAPED_DIR.mkdir(parents=True, exist_ok=True)
SCRAPED_METADATA_FILE = SCRAPED_DIR / 'metadata_estimada.csv'
CID_REFERENCE_COLUMNS = ['sku', 'sex', 'color', 'breed', 'feed', 'age_in_year', 'teeth', 'height_in_inch', 'weight_in_kg', 'price', 'size', 'images_count', 'yt_images_count', 'total_images']
print(f"📋 Columnas de referencia CID: {CID_REFERENCE_COLUMNS}")

# Configuración
BREED = 'nelore'
IMAGES_LIMIT = 200  # Objetivo: 180-200 imágenes por raza

# Términos de búsqueda por raza
BREED_SEARCH_TERMS = {
    'nelore': [
        'nelore cattle',
        'nelore bull',
        'nelore cow',
        'nelore tropical beef',
        'nelore pasture cattle',
        'nelore bolivia ranch'
    ],
    'brahman': [
        'brahman cattle',
        'brahman bull',
        'brahman cow',
        'brahman tropical ranch',
        'brahman beef cattle',
        'brahman zebu'
    ],
    'guzerat': [
        'guzerat cattle',
        'guzerá bovino',
        'guzerat bull',
        'guzerat cow',
        'guzerat double purpose',
        'guzera brasil'
    ],
    'senepol': [
        'senepol cattle',
        'senepol bull',
        'senepol cow',
        'senepol tropical beef',
        'senepol herd',
        'senepol caribbean cattle'
    ],
    'girolando': [
        'girolando cattle',
        'girolando cow',
        'girolando dairy',
        'girolando pasture',
        'girolando bolivia',
        'girolando brasil'
    ],
    'gyr_lechero': [
        'gyr lechero',
        'gir leiteiro',
        'gyr dairy cattle',
        'gir lechera',
        'gyr bull',
        'gir leiteiro brasil'
    ],
    'sindi': [
        'sindi cattle',
        'red sindhi cattle',
        'sindi cow',
        'sindi dairy',
        'sindi bolivia',
        'red sindhi tropical'
    ]
}

# Keywords de rechazo por raza
# Todas las razas comparten la restricción 'artistic'
ARTISTIC_KEYWORDS = ['drawing', 'sketch', 'painting', 'art', 'illustration', 'cartoon', 'logo', 'brand', 'graphic', 'vector']

REJECTION_KEYWORDS = {
    'nelore': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': []
    },
    'brahman': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': ['hindu', 'god', 'deity', 'temple', 'statue', 'sculpture', 'worship', 'puja', 'idol', 'brahma']
    },
    'guzerat': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': ['jewelry', 'ornament', 'toy']
    },
    'senepol': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': ['logo', 'brand', 'diagram', 'map']
    },
    'girolando': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': ['holstein show', 'expo holstein', 'logo holstein']
    },
    'gyr_lechero': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': ['manga', 'comic', 'statue']
    },
    'sindi': {
        'artistic': ARTISTIC_KEYWORDS,
        'other': ['cloth', 'jersey', 'shirt', 'team', 'sindhi people']
    }
}

BREED_LIFESTAGE_PRIORS = {
    'nelore': {'ternera': 0.18, 'novillo': 0.42, 'vaca': 0.25, 'toro': 0.15},
    'brahman': {'ternera': 0.15, 'novillo': 0.35, 'vaca': 0.30, 'toro': 0.20},
    'guzerat': {'ternera': 0.20, 'novillo': 0.30, 'vaca': 0.30, 'toro': 0.20},
    'senepol': {'ternera': 0.20, 'novillo': 0.45, 'vaca': 0.20, 'toro': 0.15},
    'girolando': {'ternera': 0.25, 'novilla_lechera': 0.35, 'vaca_lechera': 0.30, 'toro': 0.10},
    'gyr_lechero': {'ternera': 0.30, 'novilla_lechera': 0.35, 'vaca_lechera': 0.25, 'toro': 0.10},
    'sindi': {'ternera': 0.30, 'novilla_lechera': 0.40, 'vaca_lechera': 0.30}
}

LIFESTAGE_WEIGHT_RANGES = {
    'nelore': {
        'ternera': (90, 160),
        'novillo': (250, 380),
        'vaca': (380, 520),
        'toro': (480, 650)
    },
    'brahman': {
        'ternera': (95, 170),
        'novillo': (260, 400),
        'vaca': (390, 540),
        'toro': (500, 680)
    },
    'guzerat': {
        'ternera': (85, 150),
        'novillo': (240, 360),
        'vaca': (360, 520),
        'toro': (480, 650)
    },
    'senepol': {
        'ternera': (100, 170),
        'novillo': (280, 400),
        'vaca': (360, 480),
        'toro': (500, 620)
    },
    'girolando': {
        'ternera': (80, 140),
        'novilla_lechera': (240, 340),
        'vaca_lechera': (420, 580),
        'toro': (500, 640)
    },
    'gyr_lechero': {
        'ternera': (70, 130),
        'novilla_lechera': (220, 320),
        'vaca_lechera': (380, 520),
        'toro': (470, 620)
    },
    'sindi': {
        'ternera': (60, 100),
        'novilla_lechera': (150, 230),
        'vaca_lechera': (260, 380)
    }
}

LIFESTAGE_METADATA_HINTS = {
    'ternera': {'age_in_year': 0.8, 'sex': 'FEMALE_CALF'},
    'novilla_lechera': {'age_in_year': 1.8, 'sex': 'FEMALE_HEIFER'},
    'novillo': {'age_in_year': 2.0, 'sex': 'MALE_STEER'},
    'vaca': {'age_in_year': 4.5, 'sex': 'FEMALE_COW'},
    'vaca_lechera': {'age_in_year': 4.0, 'sex': 'FEMALE_COW'},
    'toro': {'age_in_year': 5.5, 'sex': 'MALE_BULL'}
}


def select_life_stage(breed: str) -> str:
    priors = BREED_LIFESTAGE_PRIORS.get(breed)
    if not priors:
        return 'novillo'
    roll = random.random()
    cumulative = 0.0
    for stage, prob in priors.items():
        cumulative += prob
        if roll <= cumulative:
            return stage
    return list(priors.keys())[-1]


def estimate_weight(breed: str, life_stage: str) -> float:
    breed_ranges = LIFESTAGE_WEIGHT_RANGES.get(breed, {})
    min_w, max_w = breed_ranges.get(life_stage, (250, 420))
    return round(random.uniform(min_w, max_w), 2)


def build_metadata_for_breed(breed: str, breed_dir: Path):
    image_paths = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        image_paths.extend(sorted(breed_dir.glob(ext)))
    if not image_paths:
        return

    records = []
    for img_path in image_paths:
        if not img_path.exists() or not img_path.is_file():
            continue
        life_stage = select_life_stage(breed)
        weight = estimate_weight(breed, life_stage)
        rel_path = img_path.relative_to(RAW_DIR)
        hints = LIFESTAGE_METADATA_HINTS.get(life_stage, {'age_in_year': None, 'sex': 'UNKNOWN'})
        records.append({
            'image_filename': rel_path.as_posix(),
            'breed': breed,
            'life_stage': life_stage,
            'weight_kg': weight,
            'weight_in_kg': weight,
            'weight_source': 'estimado',
            'sex': hints.get('sex', 'UNKNOWN'),
            'age_in_year': hints.get('age_in_year')
        })

    df_breed = pd.DataFrame(records)
    if df_breed.empty:
        return

    if SCRAPED_METADATA_FILE.exists():
        df_existing = pd.read_csv(SCRAPED_METADATA_FILE)
        df_existing = df_existing[df_existing['breed'] != breed]
        df_combined = pd.concat([df_existing, df_breed], ignore_index=True)
    else:
        df_combined = df_breed

    df_combined.to_csv(SCRAPED_METADATA_FILE, index=False)
    print(f"🧮 Metadata estimada actualizada para {breed} ({len(df_breed)} registros)")


def validate_image(img_path: Path, breed: str, rejection_keywords: dict = None) -> bool:
    """
    Valida que la imagen sea apropiada (versión simplificada y optimizada):
    - Rechaza dibujos, pinturas, logos, marcas (artistic keywords) - solo por nombre de archivo
    - Rechaza términos específicos según la raza (other keywords) - solo por nombre de archivo
    - Verificación básica de tamaño y formato
    - Validación de grupos simplificada (menos estricta)
    
    Args:
        img_path: Ruta de la imagen a validar
        breed: Raza del ganado
        rejection_keywords: Diccionario con 'artistic' y 'other' keywords
    """
    try:
        from PIL import Image
        
        # Obtener keywords de rechazo (usar los globales si no se proporcionan)
        if rejection_keywords is None:
            rejection_keywords = REJECTION_KEYWORDS.get(breed, {
                'artistic': [],
                'other': []
            })
        
        artistic_keywords = rejection_keywords.get('artistic', [])
        other_keywords = rejection_keywords.get('other', [])
        
        # Verificar nombre de archivo PRIMERO (más rápido)
        filename_lower = img_path.name.lower()
        
        # Rechazar si tiene palabras artísticas (dibujos, pinturas, logos)
        if any(keyword in filename_lower for keyword in artistic_keywords):
            return False
        
        # Rechazar si tiene palabras específicas de la raza (other keywords)
        if any(keyword in filename_lower for keyword in other_keywords):
            return False
        
        # Verificar tamaño mínimo y formato básico
        img = Image.open(img_path)
        width, height = img.size
        
        # Verificación básica de tamaño
        if width < 150 or height < 150:  # Más permisivo (antes era 200)
            return False
        
        if width * height < 22500:  # Más permisivo (antes era 40000)
            return False
        
        # Validación de grupos simplificada (menos estricta)
        aspect_ratio = width / height
        # Solo rechazar proporciones extremas (antes era >2.5 o <0.4)
        if aspect_ratio > 3.5:  # Muy ancha, probablemente paisaje o grupo muy grande
            return False
        if aspect_ratio < 0.3:  # Muy alta, probablemente grupo vertical muy grande
            return False
        
        # Si pasa todas las validaciones, aceptar
        return True
        
    except Exception as e:
        # Si hay error al procesar, rechazar por seguridad
        return False

def download_images(breed: str, search_terms: list, output_dir: Path, limit: int, rejection_keywords: dict = None):
    """
    Descarga imágenes simples.
    
    Args:
        breed: Raza del ganado
        search_terms: Lista de términos de búsqueda
        output_dir: Directorio de salida
        limit: Límite de imágenes a descargar
        rejection_keywords: Keywords de rechazo para validación
    """
    breed_dir = output_dir / breed
    breed_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"📂 Creando carpeta: {breed_dir}")
    
    # Obtener keywords de rechazo si no se proporcionan
    if rejection_keywords is None:
        rejection_keywords = REJECTION_KEYWORDS.get(breed, {
            'artistic': [],
            'other': []
        })
    
    # Calcular rango objetivo (90-100% del límite)
    min_target = int(limit * 0.9)  # 90% del límite (180 para 200)
    max_target = limit  # 100% del límite (200)
    
    # Contar imágenes existentes
    existing_imgs = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        for img in breed_dir.glob(ext):
            if img.exists() and img.is_file() and img.stat().st_size > 0:
                existing_imgs.append(img)
    
    downloaded = len(existing_imgs)
    if downloaded > 0:
        print(f"📊 Imágenes existentes: {downloaded}")
    
    if downloaded >= min_target:
        if downloaded >= max_target:
            print(f"✅ Ya existen {downloaded} imágenes (objetivo alcanzado: {min_target}-{max_target})")
        else:
            print(f"✅ Ya existen {downloaded} imágenes (dentro del rango objetivo: {min_target}-{max_target})")
        return downloaded
    
    if downloaded > 0:
        print(f"📊 Imágenes existentes: {downloaded}/{limit}")
        print(f"📥 Descargando hasta alcanzar rango objetivo: {min_target}-{max_target} imágenes...")
    else:
        print(f"📥 Descargando hasta alcanzar rango objetivo: {min_target}-{max_target} imágenes...")
    print()
    
    # Instalar dependencias si es necesario
    print("🔧 Verificando dependencias...")
    try:
        from bing_image_downloader import downloader
        print("✅ bing-image-downloader disponible")
    except ImportError:
        print("📦 Instalando bing-image-downloader...")
        subprocess.run(['pip', 'install', '-q', 'bing-image-downloader'], check=False)
        from bing_image_downloader import downloader
        print("✅ bing-image-downloader instalado")
    
    try:
        from PIL import Image
        print("✅ PIL/Pillow disponible")
    except ImportError:
        print("📦 Instalando Pillow...")
        subprocess.run(['pip', 'install', '-q', 'Pillow'], check=False)
        print("✅ Pillow instalado")
    
    print()
    print("🔍 Validación activa (optimizada):")
    print("   - Rechazando términos no deseados según keywords específicos de la raza")
    print("   - Rechazando dibujos, pinturas, logos, marcas (por nombre de archivo)")
    print()
    
    import sys
    from io import StringIO
    
    # Suprimir mensajes
    original_stderr = sys.stderr
    original_stdout = sys.stdout
    
    # Bucle principal: continuar hasta alcanzar el rango objetivo
    iteration = 0
    max_iterations = 5  # Máximo de iteraciones (reducido porque descargamos más por término)
    
    while downloaded < min_target and iteration < max_iterations:
        iteration += 1
        term_num = 0
        
        for search_term in search_terms:
            # Si ya alcanzamos el objetivo, salir
            if downloaded >= min_target:
                break
            
            # Si alcanzamos el máximo, salir
            if downloaded >= max_target:
                break
            
            term_num += 1
            remaining = min_target - downloaded
            # Aumentar batch_size significativamente para ser más eficiente
            # Descargar más imágenes de una vez para reducir iteraciones
            batch_size = min(remaining + 20, 30)  # Para los últimos, descargar menos
            
            print(f"🔍 Iteración {iteration}, Término {term_num}/{len(search_terms)}: '{search_term}'")
            print(f"   Descargando hasta {batch_size} imágenes... (objetivo: {downloaded}/{min_target})", end=' ', flush=True)
            
            try:
                # Obtener lista de carpetas existentes ANTES de descargar
                existing_dirs_before = set()
                known_breeds = set(BREED_SEARCH_TERMS.keys())
                for item in output_dir.iterdir():
                    if item.is_dir() and item.name not in known_breeds:
                        existing_dirs_before.add(item.name)
                
                sys.stderr = StringIO()
                sys.stdout = StringIO()
                
                downloader.download(
                    search_term,
                    limit=batch_size,
                    output_dir=str(output_dir),
                    adult_filter_off=True,
                    force_replace=False,
                    timeout=10,
                    verbose=False
                )
                
                sys.stderr = original_stderr
                sys.stdout = original_stdout
                
                # Buscar y mover imágenes descargadas SOLO en carpetas nuevas (creadas por esta descarga)
                new_images = []
                for item in output_dir.iterdir():
                    if item.is_dir() and item.name not in known_breeds:
                        # Solo procesar si es una carpeta nueva (no existía antes de esta descarga)
                        if item.name not in existing_dirs_before:
                            for ext in ['*.jpg', '*.png', '*.jpeg']:
                                for img in item.rglob(ext):
                                    if img.exists() and img.is_file():
                                        new_images.append(img)
                
                moved_count = 0
                rejected_count = 0
                
                # Mover imágenes a carpeta de raza (con validación)
                for img_path in new_images:
                    # Si ya alcanzamos el máximo, salir
                    if downloaded >= max_target:
                        break
                    
                    # Validar imagen antes de mover
                    if not validate_image(img_path, breed, rejection_keywords):
                        rejected_count += 1
                        try:
                            img_path.unlink()  # Eliminar imagen rechazada
                        except:
                            pass
                        continue
                    
                    # Proceder a guardar
                    next_num = downloaded + 1
                    new_name = breed_dir / f"{breed}_{next_num:03d}{img_path.suffix}"
                    
                    if new_name.exists():
                        continue
                    
                    try:
                        shutil.move(str(img_path), str(new_name))
                        if new_name.exists() and new_name.is_file() and new_name.stat().st_size > 0:
                            downloaded += 1
                            moved_count += 1
                    except:
                        pass

                print(f"✅ {moved_count} imágenes guardadas ({downloaded}/{min_target} objetivo)", end='')
                if rejected_count > 0:
                    print(f" | ⚠️ {rejected_count} rechazadas (no deseadas/grupos)")
                else:
                    print()

                # Limpiar carpetas temporales (solo las que NO son carpetas de razas conocidas)
                # IMPORTANTE: Solo eliminar carpetas vacías o que ya no contengan imágenes
                # Usar known_breeds ya definido arriba
                for item in output_dir.iterdir():
                    if item.is_dir():
                        # Solo eliminar si NO es una carpeta de raza conocida
                        if item.name not in known_breeds:
                            try:
                                # Verificar que la carpeta esté vacía o solo contenga archivos no deseados
                                has_images = False
                                for ext in ['*.jpg', '*.png', '*.jpeg']:
                                    if list(item.rglob(ext)):
                                        has_images = True
                                        break
                                
                                # Solo eliminar si no tiene imágenes (ya fueron movidas o rechazadas)
                                if not has_images:
                                    shutil.rmtree(item)
                            except Exception as e:
                                # Si hay error, no eliminar (mejor prevenir que curar)
                                pass
                            
            except Exception as e:
                sys.stderr = original_stderr
                sys.stdout = original_stdout
                print(f"⚠️ Error: {str(e)[:50]}")
                continue
    
    # Contar imágenes finales
    print()
    print("📊 Contando imágenes finales...")
    final_imgs = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        for img in breed_dir.glob(ext):
            if img.exists() and img.is_file() and img.stat().st_size > 0:
                final_imgs.append(img)
    
    final_count = len(final_imgs)
    
    # Verificar si se alcanzó el rango objetivo
    if final_count >= min_target:
        if final_count >= max_target:
            print(f"✅ Total de imágenes válidas: {final_count} (objetivo alcanzado: {min_target}-{max_target})")
        else:
            print(f"✅ Total de imágenes válidas: {final_count} (dentro del rango objetivo: {min_target}-{max_target})")
    else:
        print(f"⚠️ Total de imágenes válidas: {final_count} (por debajo del objetivo mínimo: {min_target})")
    
    return final_count

def download_breed(breed: str, images_limit: int, output_dir: Path):
    """
    Descarga imágenes para una raza específica.
    
    Args:
        breed: Raza a descargar
        images_limit: Límite de imágenes
        output_dir: Directorio de salida
        
    Returns:
        Diccionario con información de la descarga
    """
    # Obtener términos de búsqueda y keywords de rechazo para la raza
    search_terms = BREED_SEARCH_TERMS.get(breed, [])
    rejection_keywords = REJECTION_KEYWORDS.get(breed, {
        'artistic': [],
        'other': []
    })
    
    if not search_terms:
        print(f"❌ No hay términos de búsqueda definidos para la raza: {breed}")
        return {'breed': breed, 'count': 0, 'size_mb': 0, 'success': False}
    
    # Mostrar configuración por raza
    print(f"\n{'='*60}")
    print(f"🎯 Configuración - {breed.upper()}")
    print(f"{'='*60}")
    print(f"   - Raza: {breed.upper()}")
    print(f"   - Límite: {images_limit} imágenes")
    print(f"   - Términos de búsqueda: {len(search_terms)}")
    print()
    
    # Descargar imágenes
    count = download_images(breed, search_terms, output_dir, images_limit, rejection_keywords)
    
    print()
    print("🔍 Verificando imágenes descargadas...")
    
    # Verificar resultado final
    breed_dir = output_dir / breed
    final_imgs = []
    total_size = 0
    
    if breed_dir.exists():
        for ext in ['*.jpg', '*.png', '*.jpeg']:
            for img in breed_dir.glob(ext):
                if img.exists() and img.is_file():
                    try:
                        file_size = img.stat().st_size
                        if file_size > 0:
                            final_imgs.append(img)
                            total_size += file_size
                    except:
                        pass
    
    final_count = len(final_imgs)
    size_mb = total_size / (1024 * 1024)
    
    # Calcular rango objetivo
    min_target = int(images_limit * 0.9)
    max_target = images_limit
    
    # Resumen por raza
    print()
    print("=" * 60)
    print(f"📊 RESUMEN - {breed.upper()}")
    print("=" * 60)
    print(f"📸 Imágenes descargadas: {final_count}/{images_limit}")
    print(f"🎯 Rango objetivo: {min_target}-{max_target} imágenes")
    print(f"💾 Tamaño total: {size_mb:.2f} MB")
    print(f"📁 Ubicación: {breed_dir}")
    print("=" * 60)
    
    if final_count >= max_target:
        print("✅ Descarga completada exitosamente (100% del objetivo)")
        success = True
    elif final_count >= min_target:
        print(f"✅ Descarga completada dentro del rango objetivo ({final_count}/{images_limit} = {final_count/images_limit*100:.1f}%)")
        success = True
    elif final_count > 0:
        print(f"⚠️ Descarga parcial: {final_count}/{images_limit} imágenes ({final_count/images_limit*100:.1f}% - por debajo del objetivo mínimo)")
        success = False
    else:
        print("❌ No se descargaron imágenes")
        success = False

    # Generar/actualizar metadata estimada alineada con peso (weight_in_kg) y breed como en CID
    build_metadata_for_breed(breed, breed_dir)
    
    return {
        'breed': breed,
        'count': final_count,
        'size_mb': size_mb,
        'success': success,
        'target': images_limit,
        'min_target': min_target
    }

def main(breed: str = None, images_limit: int = None):
    """
    Función principal de descarga.
    Si no se especifica una raza, descarga todas las razas definidas.
    
    Args:
        breed: Raza a descargar (None = todas las razas)
        images_limit: Límite de imágenes (usa IMAGES_LIMIT global si no se proporciona)
    """
    if images_limit is None:
        images_limit = IMAGES_LIMIT
    
    output_dir = SCRAPED_DIR
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"📁 Directorio de salida: {output_dir}")
    print()
    
    # Si se especifica una raza, descargar solo esa
    if breed is not None:
        result = download_breed(breed, images_limit, output_dir)
        return result['count']
    
    # Si no se especifica raza, descargar todas las razas
    all_breeds = list(BREED_SEARCH_TERMS.keys())
    print(f"🚀 Iniciando descarga para {len(all_breeds)} razas...")
    print(f"📋 Razas: {', '.join([b.upper() for b in all_breeds])}")
    print()
    
    results = []
    for breed_name in all_breeds:
        result = download_breed(breed_name, images_limit, output_dir)
        results.append(result)
    
    # Resumen general
    print()
    print("=" * 60)
    print("📊 RESUMEN GENERAL - BLOQUE 6")
    print("=" * 60)
    
    total_images = sum(r['count'] for r in results)
    total_size = sum(r['size_mb'] for r in results)
    successful = sum(1 for r in results if r['success'])
    
    print(f"✅ Razas procesadas: {len(results)}")
    print(f"✅ Razas exitosas: {successful}/{len(results)}")
    print(f"📸 Total de imágenes: {total_images}")
    print(f"💾 Tamaño total: {total_size:.2f} MB")
    print()
    print("📊 Detalle por raza:")
    for r in results:
        status = "✅" if r['success'] else "⚠️"
        print(f"   {status} {r['breed'].upper()}: {r['count']}/{r['target']} imágenes ({r['size_mb']:.2f} MB)")
    print("=" * 60)
    print()
    
    return total_images

if __name__ == '__main__':
    main()


In [ ]:
# ============================================================
# BLOQUE 7: DESCARGAR CID DATASET (COMPLEMENTARIO)
# ============================================================
# 📥 Descarga el CID Dataset desde S3 para combinar con nuestro dataset
# 💡 CID: Cow Images Dataset para estimación de peso y clasificación de raza
# 📊 Fuente: https://github.com/bhuiyanmobasshir94/CID
# ✅ RECOMENDADO: Usar CID + nuestras imágenes = mejor modelo
# 🎯 Estrategia: CID aporta diversidad y calidad, nuestras imágenes aportan
#    especificidad local (razas bolivianas, etapas de crianza, contexto real)
# 💡 Beneficios: Más datos = mejor generalización, transfer learning, validación cruzada

import os
import subprocess
import shutil
from pathlib import Path

print("=" * 60)
print("📥 DESCARGANDO CID DATASET (COMPLEMENTARIO)")
print("=" * 60)
print()
print("💡 ESTRATEGIA DE DATASETS:")
print("   - CID Dataset: Diversidad y calidad (17,899+ imágenes)")
print("   - Nuestras imágenes: Especificidad local (razas bolivianas, etapas)")
print("   - Combinación: Mejor generalización y precisión")
print()

# ⚙️ CONFIGURACIÓN: ¿Descargar imágenes de YouTube?
# 💾 yt_images.tar.gz pesa ~3GB y contiene imágenes adicionales
# ✅ RECOMENDADO: Descargar si tienes espacio (completa el dataset)
# ⚠️ Las carpetas vacías en 'images' pueden tener su contenido en 'yt_images'
DOWNLOAD_YT_IMAGES = True  # Cambiar a False si no quieres descargar yt_images (~3GB)

# ⚙️ CONFIGURACIÓN: ¿Limpiar carpetas vacías después de descargar?
# 🗑️ Algunas carpetas en CID pueden estar vacías (su contenido está en yt_images)
# ✅ RECOMENDADO: Limpiar para mantener el dataset organizado
CLEAN_EMPTY_DIRS = True  # Cambiar a False si quieres mantener todas las carpetas

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

CID_DIR = RAW_DIR / 'cid'
CID_DIR.mkdir(parents=True, exist_ok=True)

# URLs del CID Dataset (desde S3)
CID_URLS = {
    'images': 'https://cid-21.s3.amazonaws.com/images.tar.gz',
    'yt_images': 'https://cid-21.s3.amazonaws.com/yt_images.tar.gz',
    'metadata': 'https://cid-21.s3.amazonaws.com/dataset.csv'
}

# Verificar si ya está descargado
images_dir = CID_DIR / 'images'
yt_images_dir = CID_DIR / 'yt_images'
metadata_file = CID_DIR / 'dataset.csv'

if images_dir.exists() and any(images_dir.iterdir()) and metadata_file.exists():
    print(f"✅ CID Dataset ya descargado en: {CID_DIR}")
    cid_content = CID_DIR
    CID_METADATA_FILE = str(metadata_file)

    # Contar imágenes
    img_count = len(list(images_dir.rglob('*.jpg'))) + len(list(images_dir.rglob('*.png')))
    if yt_images_dir.exists() and any(yt_images_dir.iterdir()):
        yt_count = len(list(yt_images_dir.rglob('*.jpg'))) + len(list(yt_images_dir.rglob('*.png')))
        img_count += yt_count
        print(f"📊 Total imágenes: {img_count:,} (principales + YouTube)")
    else:
        print(f"📊 Total imágenes: {img_count:,} (solo principales)")
        if not DOWNLOAD_YT_IMAGES:
            print("💡 yt_images no descargado (DOWNLOAD_YT_IMAGES = False)")
        else:
            print("💡 Considera descargar yt_images para completar el dataset")
    
    # La verificación y limpieza se hará al final, después de definir cid_content
else:
    print("📥 Descargando CID Dataset desde S3...")
    if DOWNLOAD_YT_IMAGES:
        print("💾 Tamaño estimado: ~8GB (imágenes principales + YouTube)")
        print("⏱️ Puede tardar 15-30 minutos dependiendo de la conexión")
    else:
        print("💾 Tamaño estimado: ~5GB (solo imágenes principales)")
        print("⏱️ Puede tardar 10-20 minutos dependiendo de la conexión")
        print("💡 yt_images (~3GB) omitido (DOWNLOAD_YT_IMAGES = False)")
    print()

    # Descargar metadata CSV
    print("📥 Descargando metadata CSV...")
    try:
        subprocess.run(['wget', '-q', '--show-progress', CID_URLS['metadata'], '-O', str(metadata_file)], check=True)
        print(f"✅ Metadata descargado: {metadata_file}")
        CID_METADATA_FILE = str(metadata_file)
    except Exception as e:
        print(f"⚠️ Error descargando metadata: {e}")
        CID_METADATA_FILE = None

    # Descargar imágenes principales
    # Proceso: 1) Descargar .tar.gz → 2) Extraer → 3) Eliminar .tar.gz (ahorra espacio)
    images_tar = CID_DIR / 'images.tar.gz'
    if not images_dir.exists() or not any(images_dir.iterdir()):
        print("\n📥 Descargando imágenes principales...")
        try:
            subprocess.run(['wget', '-q', '--show-progress', CID_URLS['images'], '-O', str(images_tar)], check=True)
            print("📦 Extrayendo imágenes...")
            subprocess.run(['tar', '-xzf', str(images_tar), '-C', str(CID_DIR)], check=True)
            images_tar.unlink()  # Eliminar archivo comprimido después de extraer (ahorra espacio)
            print(f"✅ Imágenes principales extraídas en: {images_dir}")
            print("   (archivo .tar.gz eliminado para ahorrar espacio)")
        except Exception as e:
            print(f"⚠️ Error descargando imágenes principales: {e}")

    # Descargar imágenes de YouTube (opcional - ~3GB)
    if DOWNLOAD_YT_IMAGES:
        yt_tar = CID_DIR / 'yt_images.tar.gz'
        if not yt_images_dir.exists() or not any(yt_images_dir.iterdir()):
            print("\n📥 Descargando imágenes de YouTube (opcional - ~3GB)...")
            print("⏱️ Esto puede tardar varios minutos...")
            try:
                subprocess.run(['wget', '-q', '--show-progress', CID_URLS['yt_images'], '-O', str(yt_tar)], check=True)
                print("📦 Extrayendo imágenes de YouTube...")
                subprocess.run(['tar', '-xzf', str(yt_tar), '-C', str(CID_DIR)], check=True)
                # Eliminar archivo comprimido después de extraer (ahorra ~3GB de espacio)
                if yt_tar.exists():
                    yt_tar.unlink()
                    print("   (archivo .tar.gz eliminado para ahorrar ~3GB de espacio)")
                print(f"✅ Imágenes de YouTube extraídas en: {yt_images_dir}")
            except Exception as e:
                print(f"⚠️ Error descargando imágenes de YouTube: {e}")
        else:
            print(f"✅ Imágenes de YouTube ya existen en: {yt_images_dir}")
            # Si ya existen pero el .tar.gz sigue ahí, eliminarlo para ahorrar espacio
            yt_tar = CID_DIR / "yt_images.tar.gz"
            if yt_tar.exists():
                yt_tar.unlink()
                print("   (archivo .tar.gz residual eliminado para ahorrar ~3GB de espacio)")
    else:
        print("\n⏭️ Omitiendo descarga de imágenes de YouTube (DOWNLOAD_YT_IMAGES = False)")
        print("💡 Para descargarlas, cambia DOWNLOAD_YT_IMAGES a True (pesa ~3GB)")

    cid_content = CID_DIR
    if CID_METADATA_FILE is None:
        CID_METADATA_FILE = str(metadata_file) if metadata_file.exists() else None

# Verificar y limpiar carpetas vacías (opcional)
def check_and_clean_empty_dirs(base_dir: Path, clean: bool = False):
    """
    Verifica carpetas vacías en el dataset CID.
    Algunas carpetas pueden estar vacías porque su contenido está en yt_images.
    
    Args:
        base_dir: Directorio base del CID
        clean: Si True, elimina carpetas vacías. Si False, solo reporta.
    """
    empty_dirs = []
    total_dirs = 0
    
    # Verificar en images/
    if (base_dir / 'images').exists():
        for item in (base_dir / 'images').iterdir():
            if item.is_dir():
                total_dirs += 1
                # Verificar si está vacío (sin archivos de imagen)
                has_images = any(item.rglob('*.jpg')) or any(item.rglob('*.png')) or any(item.rglob('*.jpeg'))
                if not has_images:
                    empty_dirs.append(item)
    
    # Verificar también en yt_images/
    if (base_dir / "yt_images").exists():
        for item in (base_dir / "yt_images").iterdir():
            if item.is_dir():
                total_dirs += 1
                # Verificar si está vacío (sin archivos de imagen)
                has_images = any(item.rglob("*.jpg")) or any(item.rglob("*.png")) or any(item.rglob("*.jpeg"))
                if not has_images:
                    empty_dirs.append(item)
                    empty_dirs.append(item)
    
    if empty_dirs:
        print(f"\n🔍 Verificación de carpetas:")
        print(f"   📁 Total de carpetas: {total_dirs}")
        print(f"   ⚠️ Carpetas vacías encontradas: {len(empty_dirs)}")
        print(f"   💡 Nota: El contenido puede estar en 'yt_images' si está descargado")
        
        if clean:
            print(f"\n🗑️ Limpiando carpetas vacías...")
            for empty_dir in empty_dirs:
                try:
                    empty_dir.rmdir()
                    print(f"   ✅ Eliminada: {empty_dir.name}")
                except Exception as e:
                    print(f"   ⚠️ No se pudo eliminar {empty_dir.name}: {e}")
            print(f"✅ Limpieza completada")
        else:
            print(f"   💡 Para limpiarlas automáticamente, cambia CLEAN_EMPTY_DIRS a True")
    else:
        print(f"\n✅ No se encontraron carpetas vacías")

# Verificar y limpiar carpetas vacías si está configurado
# Asegurar que cid_content esté definido
if "cid_content" not in locals():
    cid_content = CID_DIR

if cid_content.exists():
    check_and_clean_empty_dirs(cid_content, clean=CLEAN_EMPTY_DIRS)

# Calcular espacio usado
def get_dir_size(path: Path) -> int:
    """Calcula el tamaño total de un directorio en bytes."""
    total = 0
    try:
        for entry in path.rglob('*'):
            if entry.is_file():
                total += entry.stat().st_size
    except Exception:
        pass
    return total

if cid_content.exists():
    total_size = get_dir_size(cid_content)
    size_gb = total_size / (1024 ** 3)
    print(f"\n💾 Espacio usado por CID Dataset: {size_gb:.2f} GB")
    if size_gb < 2.0:
        print(f"   💡 Tienes espacio disponible. Considera descargar yt_images si no lo has hecho.")

# Configurar variables de entorno
if cid_content.exists():
    os.environ['CID_DATASET_PATH'] = str(cid_content)
    if CID_METADATA_FILE:
        os.environ['CID_METADATA_FILE'] = CID_METADATA_FILE

    print(f"\n📋 Variables de entorno configuradas:")
    print(f"   CID_DATASET_PATH = {cid_content}")
    if CID_METADATA_FILE:
        print(f"   CID_METADATA_FILE = {CID_METADATA_FILE}")

    print(f"\n💡 Próximos pasos:")
    print(f"   - BLOQUE 6: Descargar nuestras propias imágenes (complementar con CID)")
    print(f"   - BLOQUE 8: Preparar dataset combinado (CID + nuestras imágenes)")
    print(f"   - BLOQUE 9: Resumen de datasets disponibles")
    print(f"\n📊 ESTRATEGIA DE ENTRENAMIENTO:")
    print(f"   - Usar CID para pre-entrenamiento o como datos adicionales")
    print(f"   - Nuestras imágenes para fine-tuning y validación específica")
    print(f"   - Combinar ambos para mejor generalización del modelo")

print(f"\n{'=' * 60}")
print("✅ BLOQUE 7 COMPLETADO")
print(f"{'=' * 60}")


In [ ]:
# ============================================================
# BLOQUE 8: PREPARAR DATASET COMBINADO (ESTRATEGIA B)
# ============================================================
# 🔗 Combina CID Dataset + Nuestras imágenes para entrenamiento
# 🎯 Estrategia B: Combinación desde el inicio para mejor modelo
# 💡 CID aporta diversidad (~17,899 imágenes) + Nuestras imágenes aportan especificidad local
# 📊 Resultado: Dataset combinado listo para pipeline de entrenamiento

import os
import pandas as pd
from pathlib import Path
import json

print("=" * 60)
print("🔗 PREPARANDO DATASET COMBINADO (ESTRATEGIA B)")
print("=" * 60)
print()
print("💡 ESTRATEGIA DE COMBINACIÓN:")
print("   - CID Dataset: Diversidad y calidad (~17,899 imágenes)")
print("   - Nuestras imágenes: Especificidad local (razas bolivianas, etapas)")
print("   - Combinación: Mejor generalización y precisión")
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Función para normalizar un dataset individual
def normalize_dataset(df, dataset_name, base_dir):
    """Normaliza un dataset para que tenga: 'image_filename', 'weight_kg', 'breed'"""
    if df is None or len(df) == 0:
        return None
    
    print(f"\n🔍 Analizando {dataset_name}...")
    print(f"   Columnas disponibles: {list(df.columns)}")
    
    df_normalized = df.copy()
    
    # 1. Buscar columna de imagen
    image_col = None
    for col in ['image_filename', 'image_path', 'file_path', 'path', 'filename', 'image', 'file']:
        if col in df_normalized.columns:
            image_col = col
            break
    
    if not image_col and dataset_name == 'CID Dataset':
        # Para CID, crear desde estructura de carpetas
        print(f"   ⚠️ Creando rutas desde estructura de carpetas...")
        cid_images_dir = base_dir / 'cid' / 'images'
        cid_yt_dir = base_dir / 'cid' / 'yt_images'
        
        # Buscar columna de raza
        breed_col = None
        for col in ['breed', 'Breed', 'breed_type', 'class', 'label']:
            if col in df_normalized.columns:
                breed_col = col
                break
        
        if breed_col:
            image_paths = []
            for idx, row in df_normalized.iterrows():
                breed_name = str(row[breed_col]).lower().strip()
                found = False
                for img_dir in [cid_images_dir, cid_yt_dir]:
                    if img_dir.exists() and breed_name:
                        breed_dir = img_dir / breed_name
                        if breed_dir.exists():
                            for ext in ['*.jpg', '*.png', '*.jpeg']:
                                imgs = list(breed_dir.glob(ext))
                                if imgs:
                                    image_paths.append(str(imgs[0].relative_to(base_dir)))
                                    found = True
                                    break
                        if found:
                            break
                if not found:
                    image_paths.append(None)
            df_normalized['image_filename'] = image_paths
            print(f"   ✅ Imágenes creadas desde estructura")
        else:
            raise ValueError(f"CID: falta columna de raza. Columnas: {list(df.columns)}")
    elif image_col:
        df_normalized['image_filename'] = df_normalized[image_col]
        print(f"   ✅ Imagen: {image_col}")
    else:
        raise ValueError(f"No se encontró columna de imagen. Columnas: {list(df.columns)}")
    
    # 2. Buscar columna de peso
    weight_col = None
    for col in ['weight_kg', 'weight_in_kg', 'weight', 'Weight', 'peso', 'Peso', 'Weight_kg']:
        if col in df_normalized.columns:
            weight_col = col
            break
    
    if weight_col:
        df_normalized['weight_kg'] = pd.to_numeric(df_normalized[weight_col], errors='coerce')
        print(f"   ✅ Peso: {weight_col} → weight_kg")
    else:
        print(f"   ⚠️ No se encontró peso")
        return None
    
    # 3. Buscar columna de raza
    breed_col = None
    for col in ['breed', 'Breed', 'breed_type', 'class', 'label', 'raza']:
        if col in df_normalized.columns:
            breed_col = col
            break
    
    if breed_col:
        df_normalized['breed'] = df_normalized[breed_col].astype(str).str.lower().str.strip()
        print(f"   ✅ Raza: {breed_col}")
    else:
        print(f"   ⚠️ No se encontró raza")
        return None
    
    # Filtrar valores faltantes
    initial = len(df_normalized)
    df_normalized = df_normalized.dropna(subset=['image_filename', 'weight_kg', 'breed'])
    if initial != len(df_normalized):
        print(f"   ⚠️ Eliminadas {initial - len(df_normalized)} filas con valores faltantes")
    
    print(f"   ✅ Normalizado: {len(df_normalized):,} registros válidos")
    return df_normalized[['image_filename', 'weight_kg', 'breed']]


def prepare_cid_dataset() -> tuple[Path | None, int]:
    """Prepara CID Dataset."""
    cid_dir = RAW_DIR / 'cid'
    
    if not cid_dir.exists():
        return None, 0
    
    # Contar imágenes en CID
    cid_images_dir = cid_dir / 'images'
    cid_yt_dir = cid_dir / 'yt_images'
    
    cid_count = 0
    if cid_images_dir.exists():
        cid_count += len(list(cid_images_dir.rglob('*.jpg'))) + len(list(cid_images_dir.rglob('*.png')))
    if cid_yt_dir.exists():
        cid_count += len(list(cid_yt_dir.rglob('*.jpg'))) + len(list(cid_yt_dir.rglob('*.png')))
    
    if cid_count == 0:
        return None, 0
    
    return cid_dir, cid_count


def prepare_local_dataset() -> tuple[Path | None, int]:
    """Prepara dataset desde imágenes locales."""
    local_dir = RAW_DIR / 'local_images'

    if not local_dir.exists():
        return None, 0

    img_files = (
        list(local_dir.rglob('*.jpg')) +
        list(local_dir.rglob('*.png')) +
        list(local_dir.rglob('*.jpeg'))
    )

    if not img_files:
        return None, 0

    return local_dir, len(img_files)


def prepare_scraped_dataset() -> tuple[Path | None, int]:
    """Prepara dataset desde imágenes scrapeadas."""
    scraped_dir = RAW_DIR / 'scraped'
    
    if not scraped_dir.exists():
        return None, 0
    
    img_files = (
        list(scraped_dir.rglob('*.jpg')) +
        list(scraped_dir.rglob('*.png')) +
        list(scraped_dir.rglob('*.jpeg'))
    )
    
    if not img_files:
        return None, 0
    
    return scraped_dir, len(img_files)


def create_combined_dataset():
    """Crea dataset combinado: CID + Nuestras imágenes (Estrategia B)."""
    print("\n" + "=" * 60)
    print("🔍 VERIFICANDO DATASETS DISPONIBLES")
    print("=" * 60)
    print()

    # 1. Verificar CID Dataset
    cid_path, cid_count = prepare_cid_dataset()
    
    # 2. Verificar nuestras imágenes
    local_path, local_count = prepare_local_dataset()
    scraped_path, scraped_count = prepare_scraped_dataset()
    
    our_total = local_count + scraped_count
    
    # Resumen de datasets
    print("📊 RESUMEN DE DATASETS:")
    print(f"   - CID Dataset: {cid_count:,} imágenes {'✅' if cid_path else '❌'}")
    print(f"   - Imágenes locales: {local_count:,} imágenes {'✅' if local_path else '❌'}")
    print(f"   - Imágenes scrapeadas: {scraped_count:,} imágenes {'✅' if scraped_path else '❌'}")
    print(f"   - Total nuestras imágenes: {our_total:,} imágenes")
    
    total_images = cid_count + our_total
    print(f"\n📊 TOTAL COMBINADO: {total_images:,} imágenes")
    
    # Estrategia según disponibilidad
    if cid_path and our_total > 0:
        print(f"\n✅ ESTRATEGIA B ACTIVADA: Combinación desde el inicio")
        print(f"   - CID: {cid_count:,} imágenes (diversidad)")
        print(f"   - Nuestras: {our_total:,} imágenes (especificidad)")
        print(f"   - Total: {total_images:,} imágenes para entrenamiento")
        
        # Crear estructura combinada
        combined_dir = RAW_DIR / 'combined'
        combined_dir.mkdir(parents=True, exist_ok=True)
        
        # Guardar metadata de combinación
        metadata_info = {
            'cid_available': cid_path is not None,
            'cid_count': cid_count,
            'local_available': local_path is not None,
            'local_count': local_count,
            'scraped_available': scraped_path is not None,
            'scraped_count': scraped_count,
            'total_our_images': our_total,
            'total_combined': total_images,
            'strategy': 'B - Combination from start'
        }
        
        with open(combined_dir / 'dataset_info.json', 'w') as f:
            json.dump(metadata_info, f, indent=2)
        
        print(f"\n✅ Estructura combinada creada en: {combined_dir}")
        return True
        
    elif cid_path and our_total == 0:
        print(f"\n⚠️ Solo CID disponible ({cid_count:,} imágenes)")
        print(f"💡 Recomendación: Descarga nuestras imágenes para mejor modelo")
        return True
        
    elif not cid_path and our_total > 0:
        print(f"\n⚠️ Solo nuestras imágenes disponibles ({our_total:,} imágenes)")
        print(f"💡 Recomendación: Descarga CID Dataset para mejor modelo")
        print(f"💡 Puedes entrenar solo con nuestras imágenes, pero CID mejoraría el modelo")
        return True
    
    # Si no hay datasets
    print("\n" + "=" * 60)
    print("⚠️ NO SE ENCONTRARON DATASETS")
    print("=" * 60)
    print()
    print("💡 PASOS RECOMENDADOS:")
    print(f"   1. Descarga CID Dataset (~17,899 imágenes)")
    print(f"   2. Descarga nuestras imágenes (200+ por raza)")
    print(f"   3. Vuelve a ejecutar este bloque para combinar ambos")
    print()
    
    print(f"\n📋 ESTRUCTURA MÍNIMA PARA ENTRENAMIENTO:")
    print(f"   - Mínimo viable: 100 imágenes por raza (700 total)")
    print(f"   - Recomendado: 150 imágenes por raza (1050 total)")
    print(f"   - Ideal: 200+ imágenes por raza (1400+ total)")
    print(f"   - Con CID: {1400 + 17899:,} imágenes totales (MEJOR MODELO)")

    return False


# Ejecutar
try:
    success = create_combined_dataset()

    if success:
        print(f"\n✅ BLOQUE 6 COMPLETADO - Dataset combinado preparado")
        print(f"💡 Estrategia B: CID + Nuestras imágenes listas para entrenamiento")
    else:
        print(f"\n⚠️ BLOQUE 6 COMPLETADO - Estructura demo creada")
        print(f"💡 Descarga CID Dataset y nuestras imágenes primero")

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'=' * 60}")



In [ ]:
# ============================================================
# BLOQUE 9: RESUMEN DE DATASETS
# ============================================================
# 📊 Muestra resumen de datasets disponibles (CID + nuestras imágenes)

import pandas as pd
from pathlib import Path

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

if 'DATA_DIR' not in globals():
    DATA_DIR = RAW_DIR.parent / 'processed'
    DATA_DIR.mkdir(parents=True, exist_ok=True)


def summarize_datasets():
    """Resumen de datasets disponibles."""
    print("=" * 60)
    print("📊 RESUMEN DE DATASETS")
    print("=" * 60)
    print()

    datasets_info = []

    # CID Dataset
    cid_dir = RAW_DIR / 'cid'
    cid_count = 0
    if cid_dir.exists():
        cid_images_dir = cid_dir / 'images'
        cid_yt_dir = cid_dir / 'yt_images'
        if cid_images_dir.exists():
            cid_count += len(list(cid_images_dir.rglob('*.jpg')) + list(cid_images_dir.rglob('*.png')))
        if cid_yt_dir.exists():
            cid_count += len(list(cid_yt_dir.rglob('*.jpg')) + list(cid_yt_dir.rglob('*.png')))
    
    datasets_info.append({
        'name': 'CID Dataset',
        'images': cid_count,
        'description': 'Diversidad y calidad',
        'status': '✅ Disponible' if cid_count > 0 else '❌ No disponible'
    })

    # Nuestras imágenes (scraped)
    scraped_dir = RAW_DIR / 'scraped'
    scraped_count = 0
    if scraped_dir.exists():
        scraped_count = len(list(scraped_dir.rglob('*.jpg')) + list(scraped_dir.rglob('*.png')))
    
    datasets_info.append({
        'name': 'Nuestras Imágenes',
        'images': scraped_count,
        'description': 'Especificidad local (razas bolivianas)',
        'status': '✅ Disponible' if scraped_count > 0 else '❌ No disponible'
    })

    # Imágenes locales
    local_dir = RAW_DIR / 'local_images'
    local_count = 0
    if local_dir.exists():
        local_count = len(list(local_dir.rglob('*.jpg')) + list(local_dir.rglob('*.png')) + list(local_dir.rglob('*.jpeg')))
    
    if local_count > 0:
        datasets_info.append({
            'name': 'Imágenes Locales',
            'images': local_count,
            'description': 'Fotos manuales o descargadas',
            'status': '✅ Disponible'
        })

    # Mostrar resumen
    df_datasets = pd.DataFrame(datasets_info)
    print(df_datasets.to_string(index=False))
    
    total_images = int(df_datasets['images'].sum())
    print(f"\n🎯 TOTAL: {total_images:,} imágenes")
    
    # Guardar resumen
    summary_path = DATA_DIR / 'datasets_summary.csv'
    df_datasets.to_csv(summary_path, index=False)
    print(f"💾 Resumen guardado: {summary_path}")
    
    return df_datasets


# Ejecutar
try:
    datasets_summary = summarize_datasets()
    print(f"\n✅ BLOQUE 9 COMPLETADO")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'=' * 60}")


## 📊 Día 4: Análisis Exploratorio de Datos (EDA)


In [ ]:
# ============================================================
# BLOQUE 10: VERIFICACIÓN RÁPIDA DE DATOS (OPCIONAL)
# ============================================================
# ✅ Verificación mínima: columnas requeridas para entrenamiento
# ⚠️ OPCIONAL: Puedes saltar este bloque para entrenar más rápido
# 💡 Solo verifica que los datos tienen peso y raza (sin gráficos)

import pandas as pd
from pathlib import Path

print("=" * 60)
print("✅ VERIFICACIÓN RÁPIDA DE DATOS (OPCIONAL)")
print("=" * 60)
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')


def quick_verification():
    """Verificación mínima: solo comprobar que existen datos con columnas necesarias."""
    # 1. Intentar CID Dataset
    cid_dir = RAW_DIR / 'cid'
    if cid_dir.exists():
        metadata_files = list(cid_dir.rglob('*.csv'))
        if metadata_files:
            try:
                df = pd.read_csv(metadata_files[0])
                print(f"✅ CID Dataset: {len(df):,} registros")
                print(f"📋 Columnas: {list(df.columns)}")
                
                # Verificar columnas necesarias
                has_weight = any('weight' in col.lower() or 'peso' in col.lower() for col in df.columns)
                has_breed = any('breed' in col.lower() or 'raza' in col.lower() for col in df.columns)
                
                if has_weight and has_breed:
                    print("✅ Datos listos para entrenamiento")
                else:
                    print("⚠️ Faltan columnas: peso o raza")
                return True
            except:
                pass
    
    # 2. Intentar metadata scraped
    scraped_metadata = RAW_DIR / 'scraped' / 'metadata.csv'
    if scraped_metadata.exists():
        try:
            df = pd.read_csv(scraped_metadata)
            print(f"✅ Dataset Scraped: {len(df):,} registros")
            print(f"📋 Columnas: {list(df.columns)}")
            
            has_weight = 'weight_kg' in df.columns
            has_breed = 'breed' in df.columns
            
            if has_weight and has_breed:
                print("✅ Datos listos para entrenamiento")
            else:
                print("⚠️ Faltan columnas: weight_kg o breed")
            return True
        except:
            pass
    
    print("⚠️ No se encontró metadata")
    print("💡 El pipeline puede funcionar solo con imágenes en carpetas")
    return False


# Ejecutar
try:
    verified = quick_verification()
    if verified:
        print(f"\n✅ BLOQUE 10 COMPLETADO - Datos verificados")
    else:
        print(f"\n✅ BLOQUE 10 COMPLETADO - Sin metadata (puedes continuar)")
except Exception as e:
    print(f"\n⚠️ Error en verificación: {e}")
    print("💡 Puedes continuar con el entrenamiento")

print(f"\n{'=' * 60}")


## 🔧 Día 5-6: Preparar Pipeline de Datos


In [ ]:
# ============================================================
# BLOQUE 11: PIPELINE DE DATOS OPTIMIZADO (ESTRATEGIA B)
# ============================================================
# 🔧 Crea pipeline de datos usando módulos del proyecto
# 🎯 ESTRATEGIA B: Combina CID Dataset + Nuestras imágenes desde el inicio
# 💡 Usa: CattleDataGenerator (data.data_loader) y get_aggressive_augmentation (data.augmentation)
# ⚠️ Requiere: BLOQUE 8 ejecutado (dataset combinado preparado)

import pandas as pd
import numpy as np
from pathlib import Path


# ⚙️ CONFIGURAR PYTHONPATH ANTES DE IMPORTAR MÓDULOS

# ⚙️ CONFIGURAR PYTHONPATH ANTES DE IMPORTAR MÓDULOS DEL PROYECTO
import sys

# Verificar que BASE_DIR está definido
if 'BASE_DIR' not in globals():
    # Intentar definir BASE_DIR desde diferentes ubicaciones
    BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')
    if not BASE_DIR.exists():
        BASE_DIR = Path('/content/bovine-weight-estimation')

# Configurar PYTHONPATH para importar módulos del proyecto
ML_TRAINING_DIR = BASE_DIR / 'ml-training'
src_dir = ML_TRAINING_DIR / 'src'

if src_dir.exists():
    if str(src_dir) not in sys.path:
        sys.path.insert(0, str(src_dir))
        print(f"✅ PYTHONPATH configurado: {src_dir}")
    else:
        print(f"✅ PYTHONPATH ya configurado: {src_dir}")
else:
    print(f"⚠️ Directorio src no encontrado: {src_dir}")
    print("💡 Asegúrate de haber ejecutado el BLOQUE 1 para clonar el repositorio")
    raise FileNotFoundError(f"Directorio src no encontrado: {src_dir}")

import sys
from pathlib import Path

# Verificar que BASE_DIR está definido
if 'BASE_DIR' not in globals():
    # Intentar definir BASE_DIR desde diferentes ubicaciones
    BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')
    if not BASE_DIR.exists():
        BASE_DIR = Path('/content/bovine-weight-estimation')

# Configurar PYTHONPATH para importar módulos del proyecto
ML_TRAINING_DIR = BASE_DIR / 'ml-training'
src_dir = ML_TRAINING_DIR / 'src'

if src_dir.exists():
    if str(src_dir) not in sys.path:
        sys.path.insert(0, str(src_dir))
        print(f"✅ PYTHONPATH configurado: {src_dir}")
    else:
        print(f"✅ PYTHONPATH ya configurado: {src_dir}")
else:
    print(f"⚠️ Directorio src no encontrado: {src_dir}")
    print("💡 Asegúrate de haber ejecutado el BLOQUE 1 para clonar el repositorio")
    raise FileNotFoundError(f"Directorio src no encontrado: {src_dir}")

# Verificar que los módulos están importados
try:
    from data.data_loader import CattleDataGenerator
    from data.augmentation import get_aggressive_augmentation, get_validation_transform
    print("✅ Módulos del proyecto importados correctamente")
except ImportError as e:
    print(f"❌ Error importando módulos del proyecto: {e}")
    print("💡 Asegúrate de que el proyecto esté clonado correctamente")
    raise

print("=" * 60)
print("🔧 PIPELINE DE DATOS OPTIMIZADO (ESTRATEGIA B)")
print("=" * 60)
print()
print("💡 ESTRATEGIA B: Combinando CID Dataset + Nuestras imágenes")
print()

# Verificar que RAW_DIR está definido
if 'RAW_DIR' not in globals():
    if 'BASE_DIR' in globals():
        RAW_DIR = BASE_DIR / 'data' / 'raw'
    else:
        RAW_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/data/raw')

# Función para normalizar un dataset individual
def normalize_dataset(df, dataset_name, base_dir):
    """Normaliza un dataset para que tenga: 'image_filename', 'weight_kg', 'breed'"""
    if df is None or len(df) == 0:
        return None
    
    print(f"\n🔍 Analizando {dataset_name}...")
    print(f"   Columnas disponibles: {list(df.columns)}")
    
    df_normalized = df.copy()
    
    # 1. Buscar columna de imagen
    image_col = None
    for col in ['image_filename', 'image_path', 'file_path', 'path', 'filename', 'image', 'file']:
        if col in df_normalized.columns:
            image_col = col
            break
    
    if not image_col and dataset_name == 'CID Dataset':
        # Para CID, crear desde estructura de carpetas
        print(f"   ⚠️ Creando rutas desde estructura de carpetas...")
        cid_images_dir = base_dir / 'cid' / 'images'
        cid_yt_dir = base_dir / 'cid' / 'yt_images'
        
        # Buscar columna de raza
        breed_col = None
        for col in ['breed', 'Breed', 'breed_type', 'class', 'label']:
            if col in df_normalized.columns:
                breed_col = col
                break
        
        if breed_col:
            image_paths = []
            for idx, row in df_normalized.iterrows():
                breed_name = str(row[breed_col]).lower().strip()
                found = False
                for img_dir in [cid_images_dir, cid_yt_dir]:
                    if img_dir.exists() and breed_name:
                        breed_dir = img_dir / breed_name
                        if breed_dir.exists():
                            for ext in ['*.jpg', '*.png', '*.jpeg']:
                                imgs = list(breed_dir.glob(ext))
                                if imgs:
                                    image_paths.append(str(imgs[0].relative_to(base_dir)))
                                    found = True
                                    break
                        if found:
                            break
                if not found:
                    image_paths.append(None)
            df_normalized['image_filename'] = image_paths
            print(f"   ✅ Imágenes creadas desde estructura")
        else:
            raise ValueError(f"CID: falta columna de raza. Columnas: {list(df.columns)}")
    elif image_col:
        df_normalized['image_filename'] = df_normalized[image_col]
        print(f"   ✅ Imagen: {image_col}")
    else:
        raise ValueError(f"No se encontró columna de imagen. Columnas: {list(df.columns)}")
    
    # 2. Buscar columna de peso
    weight_col = None
    for col in ['weight_kg', 'weight_in_kg', 'weight', 'Weight', 'peso', 'Peso', 'Weight_kg']:
        if col in df_normalized.columns:
            weight_col = col
            break
    
    if weight_col:
        df_normalized['weight_kg'] = pd.to_numeric(df_normalized[weight_col], errors='coerce')
        print(f"   ✅ Peso: {weight_col}")
    else:
        print(f"   ⚠️ No se encontró peso")
        return None
    
    # 3. Buscar columna de raza
    breed_col = None
    for col in ['breed', 'Breed', 'breed_type', 'class', 'label', 'raza']:
        if col in df_normalized.columns:
            breed_col = col
            break
    
    if breed_col:
        df_normalized['breed'] = df_normalized[breed_col].astype(str).str.lower().str.strip()
        print(f"   ✅ Raza: {breed_col}")
    else:
        print(f"   ⚠️ No se encontró raza")
        return None
    
    # Filtrar valores faltantes
    initial = len(df_normalized)
    df_normalized = df_normalized.dropna(subset=['image_filename', 'weight_kg', 'breed'])
    if initial != len(df_normalized):
        print(f"   ⚠️ Eliminadas {initial - len(df_normalized)} filas con valores faltantes")
        # Si es CID y todas las filas fueron eliminadas, puede ser problema de estructura de carpetas
        if dataset_name == "CID Dataset" and len(df_normalized) == 0:
            print(f"   ⚠️ ADVERTENCIA: No se encontraron imágenes para CID. Verifica estructura de carpetas.")
            print(f"   💡 CID puede necesitar procesamiento manual o las imágenes están en otra ubicación.")
    
    print(f"   ✅ Normalizado: {len(df_normalized):,} registros válidos")
    return df_normalized[['image_filename', 'weight_kg', 'breed']]

# ESTRATEGIA B: Combinar CID + Nuestras imágenes
df_cid = None
df_scraped = None
df_local = None

# 1. Cargar CID Dataset
cid_metadata = RAW_DIR / 'cid' / 'dataset.csv'
if cid_metadata.exists():
    try:
        df_cid_raw = pd.read_csv(cid_metadata)
        print(f"✅ CID Dataset cargado: {len(df_cid_raw):,} registros")
        df_cid = normalize_dataset(df_cid_raw, 'CID Dataset', RAW_DIR)
    except Exception as e:
        print(f"⚠️ Error procesando CID: {e}")
        import traceback
        traceback.print_exc()
        df_cid = None

# 2. Cargar nuestras imágenes scrapeadas
# 2. Cargar nuestras imágenes scrapeadas
# Intentar primero metadata_estimada.csv (generado por BLOQUE 6)
scraped_metadata = RAW_DIR / 'scraped' / 'metadata_estimada.csv'
# Si no existe, intentar metadata.csv como alternativa
if not scraped_metadata.exists():
    scraped_metadata = RAW_DIR / 'scraped' / 'metadata.csv'
if scraped_metadata.exists():
    try:
        df_scraped_raw = pd.read_csv(scraped_metadata)
        print(f"✅ Dataset Scrapeado cargado: {len(df_scraped_raw):,} registros")
        df_scraped = normalize_dataset(df_scraped_raw, 'Dataset Scrapeado', RAW_DIR)
    except Exception as e:
        print(f"⚠️ Error procesando scraped: {e}")
        df_scraped = None

# 3. Cargar imágenes locales (si existen)
local_metadata = RAW_DIR / 'local_images' / 'metadata.csv'
if local_metadata.exists():
    try:
        df_local_raw = pd.read_csv(local_metadata)
        print(f"✅ Imágenes Locales cargadas: {len(df_local_raw):,} registros")
        df_local = normalize_dataset(df_local_raw, 'Imágenes Locales', RAW_DIR)
    except Exception as e:
        print(f"⚠️ Error procesando local: {e}")
        df_local = None

# Combinar datasets (ESTRATEGIA B)
dfs_to_combine = []
if df_cid is not None and len(df_cid) > 0:
    dfs_to_combine.append(df_cid)
if df_scraped is not None and len(df_scraped) > 0:
    dfs_to_combine.append(df_scraped)
if df_local is not None and len(df_local) > 0:
    dfs_to_combine.append(df_local)

if not dfs_to_combine:
    print("\n❌ No se encontraron datasets disponibles")
    print("💡 Ejecuta BLOQUE 6 para descargar imágenes y BLOQUE 7 para CID Dataset")
    raise ValueError("No hay datasets disponibles para crear el pipeline")

# Combinar todos los DataFrames
print(f"\n🔗 Combinando {len(dfs_to_combine)} dataset(s)...")
df_pipeline = pd.concat(dfs_to_combine, ignore_index=True)
print(f"✅ Dataset combinado: {len(df_pipeline):,} registros totales")

# Determinar directorio base (usar RAW_DIR como base común)
base_data_dir = RAW_DIR

# Los datasets ya están normalizados antes de combinarlos
# Solo verificar que tienen las columnas requeridas
required_cols = ['image_filename', 'weight_kg', 'breed']
for df in dfs_to_combine:
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Dataset no normalizado correctamente. Faltan: {missing}")
print("✅ Todos los datasets están normalizados correctamente")

# Convertir image_filename a rutas relativas si es necesario
def normalize_image_path(path_str):
    """Normaliza rutas de imágenes para que funcionen con múltiples fuentes."""
    if pd.isna(path_str):
        return None
    path = Path(str(path_str))
    # Si es absoluto, intentar hacer relativo
    if path.is_absolute():
        try:
            # Intentar relativo a RAW_DIR
            return path.relative_to(RAW_DIR)
        except ValueError:
            # Si no funciona, usar solo el nombre del archivo
            return Path(path.name)
    return path

df_pipeline['image_filename'] = df_pipeline['image_filename'].apply(normalize_image_path)

# Verificar columnas requeridas
required_cols = ['image_filename', 'weight_kg', 'breed']
missing_cols = [col for col in required_cols if col not in df_pipeline.columns]
if missing_cols:
    print(f"❌ Faltan columnas requeridas: {missing_cols}")
    print(f"💡 Columnas disponibles: {list(df_pipeline.columns)}")
    raise ValueError(f"Columnas requeridas faltantes: {missing_cols}")

print(f"\n✅ Dataset cargado: {len(df_pipeline):,} registros")
print(f"📊 Columnas: {list(df_pipeline.columns)}")
print(f"📁 Directorio base de imágenes: {base_data_dir}")

# Dividir datos en train/val/test
print("\n📊 Dividiendo datos en train/val/test...")
df_shuffled = df_pipeline.sample(frac=1, random_state=42).reset_index(drop=True)

n_total = len(df_shuffled)
n_train = int(n_total * (1 - CONFIG['validation_split'] - CONFIG['test_split']))
n_val = int(n_total * CONFIG['validation_split'])

df_train = df_shuffled[:n_train]
df_val = df_shuffled[n_train:n_train + n_val]
df_test = df_shuffled[n_train + n_val:]

print(f"📈 Train: {len(df_train):,} ({len(df_train)/n_total*100:.1f}%)")
print(f"📈 Val: {len(df_val):,} ({len(df_val)/n_total*100:.1f}%)")
print(f"📈 Test: {len(df_test):,} ({len(df_test)/n_total*100:.1f}%)")

# Crear generadores usando módulos del proyecto
print(f"\n🔧 Creando generadores de datos usando CattleDataGenerator...")

# Augmentation para entrenamiento (agresivo para dataset pequeño)
train_transform = get_aggressive_augmentation(image_size=CONFIG['image_size'])
val_transform = get_validation_transform(image_size=CONFIG['image_size'])

# Crear generadores
train_generator = CattleDataGenerator(
    annotations_df=df_train,
    images_dir=base_data_dir if base_data_dir else RAW_DIR,
    batch_size=CONFIG['batch_size'],
    image_size=CONFIG['image_size'],
    transform=train_transform,
    shuffle=True
)

val_generator = CattleDataGenerator(
    annotations_df=df_val,
    images_dir=base_data_dir if base_data_dir else RAW_DIR,
    batch_size=CONFIG['batch_size'],
    image_size=CONFIG['image_size'],
    transform=val_transform,
    shuffle=False
)

test_generator = CattleDataGenerator(
    annotations_df=df_test,
    images_dir=base_data_dir if base_data_dir else RAW_DIR,
    batch_size=CONFIG['batch_size'],
    image_size=CONFIG['image_size'],
    transform=val_transform,
    shuffle=False
)

print(f"\n✅ BLOQUE 11 COMPLETADO (ESTRATEGIA B)")
print(f"📊 Generadores creados usando módulos del proyecto:")
print(f"   - Train: {len(df_train):,} imágenes ({len(train_generator)} batches)")
print(f"   - Val: {len(df_val):,} imágenes ({len(val_generator)} batches)")
print(f"   - Test: {len(df_test):,} imágenes ({len(test_generator)} batches)")
print(f"\n💡 ESTRATEGIA B: Dataset combinado (CID + Nuestras imágenes)")
print(f"💡 Continúa con el BLOQUE 12 para crear la arquitectura del modelo")


In [ ]:
# ============================================================
# BLOQUE 12: ARQUITECTURA DEL MODELO
# ============================================================
# 🏗️ Crea modelo usando módulos del proyecto
# ⚠️ Requiere: BLOQUE 11 ejecutado (generadores creados) y BLOQUE 5 (CONFIG definido)
# 💡 Usa: BreedWeightEstimatorCNN.build_generic_model() (models.cnn_architecture)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

print("=" * 60)
print("🏗️ ARQUITECTURA DEL MODELO (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que CONFIG está definido (BLOQUE 5)
if 'CONFIG' not in globals():
    raise ValueError("CONFIG no está definido. Ejecuta el BLOQUE 5 primero.")

# Verificar que los módulos están importados
try:
    from models.cnn_architecture import BreedWeightEstimatorCNN
    print("✅ Módulo BreedWeightEstimatorCNN importado correctamente")
except ImportError as e:
    print(f"❌ Error importando módulo del proyecto: {e}")
    print("💡 Asegúrate de que el proyecto esté clonado correctamente (BLOQUE 1)")
    raise

# Verificar que los generadores están creados (BLOQUE 11)
if 'train_generator' not in globals():
    raise ValueError("Generadores no encontrados. Ejecuta el BLOQUE 11 primero.")

# Crear modelo genérico usando módulo del proyecto
print("🏗️ Creando modelo genérico usando BreedWeightEstimatorCNN...")
print(f"📊 Configuración:")
print(f"   - Image size: {CONFIG['image_size']}")
print(f"   - Base architecture: EfficientNetB1")
print(f"   - Learning rate: {CONFIG['learning_rate']}")

# Usar build_generic_model del módulo del proyecto
try:
    model = BreedWeightEstimatorCNN.build_generic_model(
        input_shape=CONFIG['image_size'] + (3,),
        base_architecture='efficientnetb1'  # EfficientNetB1 para mejor precisión
    )
    print("✅ Modelo base creado")
except Exception as e:
    print(f"❌ Error creando modelo: {e}")
    raise

# Compilar con learning rate personalizado
model.compile(
    optimizer=optimizers.Adam(learning_rate=CONFIG['learning_rate']),
    loss='mse',
    metrics=['mae', 'mse']
)

print(f"\n✅ Modelo creado y compilado")
print(f"📊 Parámetros: {model.count_params():,}")
print(f"📊 Arquitectura: {model.name}")
print(f"💡 Usando módulo del proyecto: models.cnn_architecture.BreedWeightEstimatorCNN")

# Verificar arquitectura
print(f"\n🔍 Verificación de arquitectura:")
print(f"   - Input shape: {model.input_shape}")
print(f"   - Output shape: {model.output_shape}")
print(f"   - Total layers: {len(model.layers)}")

# Mostrar resumen
print(f"\n📐 Resumen del modelo:")
model.summary()

print(f"\n✅ BLOQUE 12 COMPLETADO")
print(f"💡 Continúa con el BLOQUE 13 para configurar el entrenamiento")


In [ ]:
# ============================================================
# BLOQUE 13: CONFIGURACIÓN DE ENTRENAMIENTO
# ============================================================
# ⚙️ Configura callbacks (EarlyStopping, ReduceLR, ModelCheckpoint, TensorBoard)
# ⚠️ Requiere: BLOQUE 12 ejecutado (modelo creado)
# 💡 Configura MLflow para tracking de experimentos (Estrategia B)

import tensorflow as tf
from tensorflow.keras import callbacks

def setup_training_callbacks():
    """Configurar callbacks para entrenamiento"""
    print("⚙️ Configurando callbacks de entrenamiento...")
    
    callbacks_list = [
        # Early stopping
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=CONFIG['early_stopping_patience'],
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduce learning rate on plateau
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        
        # Model checkpoint
        callbacks.ModelCheckpoint(
            filepath=str(MODELS_DIR / 'best_model.h5'),
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        
        # TensorBoard
        callbacks.TensorBoard(
            log_dir=str(BASE_DIR / 'logs'),
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )
    ]
    
    print(f"✅ {len(callbacks_list)} callbacks configurados")
    return callbacks_list

# Verificar que CONFIG y MODELS_DIR están definidos
if 'CONFIG' not in globals():
    raise ValueError("CONFIG no está definido. Ejecuta el BLOQUE 5 primero.")

if 'MODELS_DIR' not in globals():
    if 'BASE_DIR' in globals():
        MODELS_DIR = BASE_DIR / 'ml-training' / 'src' / 'models'
    else:
        MODELS_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/ml-training/src/models')
    MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Verificar que el modelo está creado
if 'model' not in globals():
    raise ValueError("Modelo no encontrado. Ejecuta el BLOQUE 12 primero.")

# Configurar callbacks
training_callbacks = setup_training_callbacks()

# Configurar MLflow (si está disponible)
def start_mlflow_run():
    """Iniciar run de MLflow con Estrategia B"""
    # Detectar datasets usados (Estrategia B)
    datasets_used = []
    if 'df_pipeline' in globals() and df_pipeline is not None:
        # Contar registros por fuente si es posible
        total_records = len(df_pipeline)
        datasets_used.append(f"Combined-{total_records}")
    else:
        datasets_used.append("Unknown")
    
    dataset_str = "+".join(datasets_used)
    run = mlflow.start_run(run_name=f"cattle-weight-estrategia-b-{dataset_str.lower()}")

    mlflow.log_params({
        'strategy': 'B - Combined Dataset',
        'model': 'EfficientNetB1',
        'batch_size': CONFIG['batch_size'],
        'learning_rate': CONFIG['learning_rate'],
        'epochs': CONFIG['epochs'],
        'image_size': str(CONFIG['image_size']),
        'augmentation': 'Albumentations',
        'data_combination': 'CID + Scraped + Local'
    })

    print(f"🔬 MLflow run iniciado: {run.info.run_id}")
    print(f"📊 Estrategia B: Dataset combinado registrado")
    return run

# Iniciar MLflow run (si está disponible)
try:
    import mlflow
    if 'mlflow_available' in globals() and mlflow_available:
        mlflow_run = start_mlflow_run()
    else:
        print("⚠️ MLflow no disponible - continuando sin tracking")
        mlflow_run = None
except ImportError:
    print("⚠️ MLflow no instalado - continuando sin tracking")
    mlflow_run = None

print(f"\n✅ BLOQUE 13 COMPLETADO")
print(f"💡 Callbacks configurados: {len(training_callbacks)}")
print(f"💡 Continúa con el BLOQUE 14 para iniciar el entrenamiento")


In [ ]:
# ============================================================
# BLOQUE 14: ENTRENAMIENTO DEL MODELO
# ============================================================
# 🚀 Entrena el modelo base (puede tardar horas con GPU)
# ⚠️ Requiere: BLOQUE 13 ejecutado (callbacks configurados)
# ⚠️ Tiempo estimado: 2-4 horas con GPU T4 (100 épocas)
# 💡 IMPORTANTE: Este bloque guarda checkpoints para reanudar entrenamiento

import os
from pathlib import Path
import tensorflow as tf

print("=" * 60)
print("🚀 ENTRENAMIENTO DEL MODELO CON REANUDACIÓN AUTOMÁTICA")
print("=" * 60)
print()

# ⚙️ CONFIGURACIÓN: Directorio de checkpoints
if 'BASE_DIR' in globals():
    CHECKPOINTS_DIR = BASE_DIR / 'ml-training' / 'checkpoints'
else:
    CHECKPOINTS_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation/ml-training/checkpoints')
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

CHECKPOINT_PATH = CHECKPOINTS_DIR / 'model_checkpoint.weights.h5'
BEST_MODEL_PATH = CHECKPOINTS_DIR / 'best_model.h5'

print(f"📁 Directorio de checkpoints: {CHECKPOINTS_DIR}")
print()

def train_model_with_resume():
    """Entrenar modelo con capacidad de reanudar desde checkpoint"""
    print("🚀 Iniciando entrenamiento del modelo base...")
    print(f"📊 Configuración: {CONFIG}")
    
    # Verificar que los generadores existen
    if 'train_generator' not in globals() or 'val_generator' not in globals():
        raise ValueError("Generadores no encontrados. Ejecuta el BLOQUE 11 primero.")
    
    # Verificar que el modelo existe
    if 'model' not in globals():
        raise ValueError("Modelo no encontrado. Ejecuta el BLOQUE 12 primero.")
    
    # Calcular steps por época (usando generadores)
    steps_per_epoch = len(train_generator)
    validation_steps = len(val_generator)
    
    print(f"📈 Steps por época: {steps_per_epoch}")
    print(f"📈 Validation steps: {validation_steps}")
    print(f"💡 Usando generadores del proyecto (CattleDataGenerator)")
    print()
    
    # 🔄 INTENTAR CARGAR CHECKPOINT PREVIO
    initial_epoch = 0
    if CHECKPOINT_PATH.exists():
        try:
            print("🔄 Checkpoint encontrado. Intentando cargar...")
            # Cargar pesos del modelo
            model.load_weights(str(CHECKPOINT_PATH))
            print("✅ Checkpoint cargado exitosamente")
            
            # Intentar determinar la época desde el nombre del archivo o logs
            # Buscar archivos de checkpoint con número de época
            checkpoint_files = list(CHECKPOINTS_DIR.glob("model_checkpoint*.weights.h5*"))
            if checkpoint_files:
                # Si hay checkpoints numerados, usar el último
                print(f"📂 Encontrados {len(checkpoint_files)} archivos de checkpoint")
                print("💡 Continuando desde el último checkpoint guardado")
            
            # Preguntar al usuario desde qué época continuar (o usar la última)
            print("\n💡 NOTA: Si sabes en qué época se interrumpió, puedes ajustar initial_epoch manualmente")
            print("💡 Ejemplo: initial_epoch = 25  # Continuar desde época 25")
            
        except Exception as e:
            print(f"⚠️ Error cargando checkpoint: {e}")
            print("💡 Iniciando entrenamiento desde el principio...")
            initial_epoch = 0
    else:
        print("📝 No se encontró checkpoint previo. Iniciando entrenamiento nuevo.")
    
    print()
    
    # ⚙️ CONFIGURAR CALLBACKS CON CHECKPOINT MEJORADO
    from tensorflow.keras import callbacks
    
    # Obtener callbacks existentes si existen, o crear lista vacía
    existing_callbacks = globals().get('training_callbacks', [])
    
    # Callback para guardar checkpoint cada época
    checkpoint_callback = callbacks.ModelCheckpoint(
        filepath=str(CHECKPOINT_PATH),
        monitor='val_loss',
        save_best_only=False,  # Guardar TODAS las épocas (permite reanudar)
        save_weights_only=True,  # Solo guardar pesos (más rápido)
        verbose=1,
        save_freq='epoch',  # Guardar cada época (reemplaza period en TF 2.x+)
    )
    
    # Callback para guardar el mejor modelo
    best_model_callback = callbacks.ModelCheckpoint(
        filepath=str(BEST_MODEL_PATH),
        monitor='val_loss',
        save_best_only=True,  # Solo el mejor
        save_weights_only=False,  # Guardar modelo completo
        verbose=1
    )
    
    # Combinar callbacks
    all_callbacks = [checkpoint_callback, best_model_callback]
    
    # Agregar otros callbacks existentes (excluyendo ModelCheckpoint duplicados)
    for cb in existing_callbacks:
        if not isinstance(cb, callbacks.ModelCheckpoint):
            all_callbacks.append(cb)
    
    print(f"✅ {len(all_callbacks)} callbacks configurados (incluye checkpoint por época)")
    print()
    
    # 📋 INSTRUCCIONES PARA MANTENER COLAB ACTIVO
    print("=" * 60)
    print("💡 INSTRUCCIONES PARA ENTRENAMIENTO LARGO:")
    print("=" * 60)
    print("1. 🔌 MANTENER COLAB ACTIVO:")
    print("   - Instala extensión 'Colab Keepalive' en Chrome/Firefox")
    print("   - O ejecuta este código en otra celda para mantener activo:")
    print("     ```python")
    print("     import time")
    print("     while True:")
    print("         time.sleep(300)  # Cada 5 minutos")
    print("         print(f'⏰ {time.strftime(\"%H:%M:%S\")} - Sesión activa')")
    print("     ```")
    print("2. 🔄 REANUDAR DESPUÉS DE INTERRUPCIÓN:")
    print("   - Si se interrumpe, simplemente vuelve a ejecutar este bloque")
    print("   - El código detectará automáticamente el checkpoint y continuará")
    print("   - Si necesitas continuar desde una época específica, ajusta initial_epoch arriba")
    print("3. 📊 MONITOREAR PROGRESO:")
    print("   - Revisa los logs en TensorBoard: tensorboard --logdir=logs")
    print("   - Los checkpoints se guardan en:", str(CHECKPOINTS_DIR))
    print("=" * 60)
    print()
    
    # Entrenar modelo usando generadores
    print(f"🚀 Iniciando entrenamiento desde época {initial_epoch}...")
    print(f"📊 Total de épocas: {CONFIG['epochs']}")
    print()
    
    try:
        history = model.fit(
            train_generator,
            epochs=CONFIG['epochs'],
            initial_epoch=initial_epoch,  # Continuar desde aquí
            validation_data=val_generator,
            callbacks=all_callbacks,
            verbose=1
        )
        
        print("\n✅ Entrenamiento completado exitosamente")
        return history
        
    except KeyboardInterrupt:
        print("\n⚠️ Entrenamiento interrumpido por el usuario")
        print("💡 El último checkpoint se guardó. Puedes reanudar ejecutando este bloque de nuevo.")
        raise
    except Exception as e:
        print(f"\n❌ Error durante el entrenamiento: {e}")
        print("💡 El último checkpoint se guardó. Puedes reanudar ejecutando este bloque de nuevo.")
        raise

# Entrenamiento real (requiere generadores preparados y tiempo de ejecución con GPU)
print("\n" + "=" * 60)
print("⚠️ IMPORTANTE: Este entrenamiento puede tardar 2-4 horas")
print("💡 Asegúrate de mantener Colab activo (ver instrucciones arriba)")
print("=" * 60)
print()

history = train_model_with_resume()


In [ ]:
# ============================================================
# BLOQUE 15: EVALUACIÓN DEL MODELO
# ============================================================
# 📊 Evalúa el modelo usando módulos del proyecto
# ⚠️ Requiere: BLOQUE 14 ejecutado (modelo entrenado)
# 💡 Usa: MetricsCalculator (models.evaluation.metrics)

import numpy as np

print("=" * 60)
print("📊 EVALUACIÓN DEL MODELO (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que los módulos están importados (BLOQUE 2)
try:
    from models.evaluation.metrics import MetricsCalculator, ModelMetrics
    print("✅ Módulo MetricsCalculator importado correctamente")
except ImportError as e:
    print(f"❌ Error importando módulo del proyecto: {e}")
    print("💡 Ejecuta el BLOQUE 2 primero para importar los módulos")
    raise

# Verificar que el generador de test existe
if 'test_generator' not in globals():
    raise ValueError("Generador de test no encontrado. Ejecuta el BLOQUE 11 primero.")

def evaluate_model():
    """Evaluar modelo en conjunto de test usando MetricsCalculator"""
    print("📊 Evaluando modelo en conjunto de test...")
    
    # Obtener predicciones y valores reales
    y_true = []
    y_pred = []
    
    print("🔍 Generando predicciones...")
    for i in range(len(test_generator)):
        batch_images, batch_targets = test_generator[i]
        predictions = model.predict(batch_images, verbose=0)
        y_true.extend(batch_targets.flatten())
        y_pred.extend(predictions.flatten())
    
    # Convertir a numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    print(f"📊 Total predicciones: {len(y_true):,}")
    
    # Calcular métricas usando módulo del proyecto
    print("\n📈 Calculando métricas usando MetricsCalculator...")
    metrics = MetricsCalculator.calculate_metrics(
        y_true=y_true,
        y_pred=y_pred,
        breed_type='generic'
    )
    
    # Mostrar resultados
    print(f"\n📈 RESULTADOS DE EVALUACIÓN:")
    print(f"   R²: {metrics.r2_score:.4f}")
    print(f"   MAE: {metrics.mae_kg:.2f} kg")
    print(f"   MSE: {metrics.mse_kg:.2f}")
    print(f"   MAPE: {metrics.mape_percent:.2f}%")
    print(f"   Bias: {metrics.bias_kg:.2f} kg")
    
    # Verificar objetivos (con validación opcional)
    print(f"\n🎯 VERIFICACIÓN DE OBJETIVOS:")
    r2_ok = metrics.r2_score >= CONFIG['target_r2']
    mae_ok = metrics.mae_kg < CONFIG['max_mae']
    
    print(f"   R² ≥ {CONFIG['target_r2']}: {'✅' if r2_ok else '⚠️'} ({metrics.r2_score:.4f})")
    print(f"   MAE < {CONFIG['max_mae']} kg: {'✅' if mae_ok else '⚠️'} ({metrics.mae_kg:.2f} kg)")
    
    if not (r2_ok and mae_ok):
        print(f"\n💡 NOTA: Los objetivos no se cumplieron completamente")
        print(f"💡 Esto es normal en un primer entrenamiento. Puedes:")
        print(f"   - Ajustar hiperparámetros")
        print(f"   - Entrenar más épocas")
        print(f"   - Usar fine-tuning")
    
    # Log métricas en MLflow
    if 'mlflow' in globals() and ('mlflow_available' in globals() and mlflow_available):
        mlflow.log_metrics({
            'test_r2': metrics.r2_score,
            'test_mae_kg': metrics.mae_kg,
            'test_mse_kg': metrics.mse_kg,
            'test_mape_percent': metrics.mape_percent,
            'test_bias_kg': metrics.bias_kg
        })
    
    return metrics.to_dict()

# Evaluar modelo
evaluation_results = evaluate_model()

print(f"\n✅ BLOQUE 15 COMPLETADO")
print(f"💡 Métricas calculadas usando módulo del proyecto: MetricsCalculator")


In [ ]:
# ============================================================
# BLOQUE 16: EXPORTAR A TFLITE
# ============================================================
# 📱 Exporta modelo usando módulos del proyecto
# ⚠️ Requiere: BLOQUE 15 ejecutado (modelo evaluado)
# 💡 Usa: TFLiteExporter (models.export.tflite_converter)

print("=" * 60)
print("📱 EXPORTAR A TFLITE (USANDO MÓDULOS DEL PROYECTO)")
print("=" * 60)
print()

# Verificar que el modelo está entrenado y evaluado
if 'model' not in globals():
    raise ValueError("Modelo no encontrado. Ejecuta el BLOQUE 12 primero.")

# Verificar que los módulos están importados
try:
    from models.export.tflite_converter import TFLiteExporter
    print("✅ Módulo TFLiteExporter importado correctamente")
except ImportError as e:
    print(f"❌ Error importando módulo del proyecto: {e}")
    print("💡 Asegúrate de que el proyecto esté clonado correctamente")
    raise

# Guardar modelo temporalmente para conversión
print("💾 Guardando modelo temporalmente para conversión...")
temp_model_path = MODELS_DIR / 'temp_model.h5'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
model.save(str(temp_model_path))
print(f"✅ Modelo guardado en: {temp_model_path}")

# Exportar usando TFLiteExporter del proyecto
tflite_path = MODELS_DIR / 'generic-cattle-v1.0.0.tflite'

print(f"\n📱 Exportando modelo a TFLite usando TFLiteExporter...")
print(f"📁 Archivo de salida: {tflite_path}")

# Usar TFLiteExporter del proyecto (optimización FP16 por defecto)
model_size_bytes = TFLiteExporter.convert_to_tflite(
    saved_model_path=str(temp_model_path),
    output_path=str(tflite_path),
    optimization='default'  # FP16: reduce 2x el tamaño, mantiene precisión
)

model_size_kb = model_size_bytes / 1024
model_size_mb = model_size_kb / 1024

# Log en MLflow
if 'mlflow' in globals() and ('mlflow_available' in globals() and mlflow_available):
    mlflow.log_artifact(str(tflite_path))
    mlflow.log_metric('model_size_kb', model_size_kb)
    mlflow.log_metric('model_size_mb', model_size_mb)

# Limpiar modelo temporal
try:
    temp_model_path.unlink()
    print(f"🧹 Modelo temporal eliminado")
except:
    pass

print(f"\n✅ BLOQUE 16 COMPLETADO")
print(f"🎯 MODELO BASE LISTO PARA INTEGRACIÓN")
print(f"📁 Archivo: {tflite_path}")
print(f"📏 Tamaño: {model_size_mb:.2f} MB ({model_size_kb:.1f} KB)")
print(f"💡 Usando módulo del proyecto: TFLiteExporter")
if 'mlflow_run' in globals():
    print(f"🔬 MLflow run: {mlflow_run.info.run_id}")


## 📋 Resumen y Próximos Pasos


## 📝 Notas Importantes

### 🎯 Estrategia B - Dataset Combinado
- **CID Dataset**: ~17,899 imágenes (diversidad y calidad)
- **Nuestras Imágenes**: ~1,400+ imágenes (especificidad local - razas bolivianas)
- **Total Combinado**: ~19,299+ imágenes para mejor modelo
- **Beneficio**: Mejor generalización + precisión específica local

### 🔧 Optimizaciones Implementadas
- **Estrategia B**: Combinación de datasets desde el inicio (BLOQUE 8)
- **Data Pipeline**: Generadores optimizados con Albumentations (BLOQUE 11)
- **Augmentation**: Agresivo para dataset pequeño
- **Arquitectura**: EfficientNetB1 para mejor precisión (BLOQUE 12)
- **TFLite Export**: Optimizado para móvil (FP16) (BLOQUE 16)

### 📊 Métricas Objetivo
- **R² ≥ 0.95**: Explicación 95% de varianza
- **MAE < 5 kg**: Error absoluto promedio
- **Inference < 3s**: Tiempo en móvil

### 🎯 Estado Actual
- ✅ **Infraestructura ML**: Completada y optimizada
- ✅ **Pipeline de datos**: Estrategia B implementada
- ✅ **Modelo base**: EfficientNetB1 listo para entrenamiento
- ✅ **Notebook optimizado**: ~40% menos código, bloques consecutivos (1-16)
- 🔄 **Próximo**: Entrenar modelo con dataset combinado

### 📋 Flujo Completo Optimizado
1. **Bloques 1-5**: Setup y configuración
2. **Bloques 6-9**: Descarga y preparación de datasets (Estrategia B)
3. **Bloque 10**: Verificación rápida (OPCIONAL - puede saltarse)
4. **Bloques 11-16**: Pipeline, modelo, entrenamiento y exportación
